## -2. 更新日志

+ 【2017/05/11】
    - 重新梳理前期实验结果，并整合到该份报告中
    - 前期实验报告参见同一目录下的其他以 `trial_` 开头的 `.ipynb` 文件

## -1. 备注

1. 搜索该符号以定位到报告中待完善的部分：。。。
2. 稍后可考虑将部分代码包装成 Python 脚本，以模块的形式导入，使整个 notebook 更简洁？或者不这样处理，从而使读者更方便阅读（而不用另外切换于多个页面之间）？
3. 。。。

## 0. 参考文献

1. (#miscellaneous) [20 Newsgroup Document Classification Report](http://cn-static.udacity.com/mlnd/Capstone_Poject_Sample01.pdf)
2. (#word2vec, #tensorflow) [Vector Representations of Words](https://www.tensorflow.org/tutorials/word2vec)
3. (#word2vec) [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
4. (#word2vec, #gensim) [models.word2vec – Deep learning with word2vec](https://radimrehurek.com/gensim/models/word2vec.html)
5. (#CNN, #tensorflow) [Deep MNIST for Experts](https://www.tensorflow.org/get_started/mnist/pros)
6. (#text8) [text8](http://mattmahoney.net/dc/textdata)
7. 。。。

## 1. 实验模块规划

粗略规划如下，稍后精细整理：实现下述共计 6 种「表示 + 训练」组合

表示模型 | 分类器训练法
---------------|---------------------
BOW + TF-IDF | SVM
                     | LSA
                     | LDA
Word2Vec      | SVM
                     | DNN
                     | CNN
                     
其中：
+ 名词解释
    - 表示模型
        * BOW：Bag-of-Words，词袋模型
        * TF-IDF：Term Frequency - Inverse Document Frequency，文档-逆文档频率
    - 分类器训练法
        * SVM：Support Vector Machine，支持向量机
        * LSA：潜在语义分析
        * LDA：Latent Dirichlet Allocation，隐含狄利克雷分布
        * DNN：Deep Neural Network，深度神经网络（普通的多层感知机构成的多层神经网络）
        * CNN：Convolution Neural Network，卷积神经网络
+ 训练工具
    - 表示模型
        * BOW+TF-IDF：使用 scikit-learn 建模
        * Word2Vec：使用 gensim 与 TensorFlow 建模
    - 分类器训练法
        * 传统算法：SVM, LSA, LDA 使用 scikit-learn 训练
        * 神经网络算法：DNN, CNN：使用 TensorFlow 训练

## 2. 实验流程规划

1. 模块导入
2. 数据预处理（通用预处理）
  + 对文本进行清洗，包括但不限于去除特殊符号、进行大小写转换等工作，最终使文本中只包含：由小写字母 a-z 组成的单词、单一空格
  + 不在 a-z 之间的字符将一律被转换为空格
3. 文本表示建模
  + BOW+TF-IDF 表示：
      1. 读入原始语料并保存
      2. 使用 scikit-learn，在原始语料的基础上，进行建立词袋（BOW）、计算 TF-IDF
      3. 通过使用 BOW+TF-IDF 向量表示文档中的每个词，从而表示每篇文档（包括所有语料：训练集和测试集）
      4. 每篇文档对应的标签独热（one-hot）向量化
  + Word2Vec 表示：
      1. 分别使用 gensim 和 TensorFlow 中的每一种，分别在 text8 的基础上、在待学习样本的基础上，建立词嵌入（word embedding）模型（Word2Vec）
          + 即：训练出 gensim+text8, gensim+待学习样本, TensorFlow+text8, TensorFlow+待学习样本 共计 4 种表示模型
          + 使用 Skip-Gram 方法进行建模
      2. 通过使用 Word2Vec 向量表示文档中的每个词，然后建立 2 种文档表示模型：
          + 求这些词向量的和，以求和向量表示每一篇文档；对于不在词汇表中的词，以某常量代替——具体而言，可指定为加入零向量，或在求和向量乘上某个常量系数
          + 对于上述求和向量进行求算术平均，使用算术平均向量表示每一篇文档；对于不在词汇表中的词，同上述处理方法
      3. 每篇文档对应的标签独热（one-hot）向量化
4. 分类器训练
  + 传统算法：SVM, LSA, LDA 使用 scikit-learn 训练
  + 神经网络算法：DNN, CNN：使用 TensorFlow 训练
5. 分类器评估
  + 对于传统算法：
    - 使用 sckit-learn 提供的 GridSearchCV 与 LearningCurve 方法寻找最优参数组合
    - 使用 scikit-learn 提供的 accuracy_score（查准率 P） 与 f1_score（F1 分数，同时考察了查准率 P 与查全率 R） 评估训练结果
  + 对于神经网络算法：
    - 暂定手工选择一组参数进行训练；待考察是否可使用 GridSearchCV 与 LearningCurve 进行参数组合寻找最优参数组合
    - 暂定使用手工编写的方法计算查准率 P、查全率 R、F1 分数；待考察是否可对数据格式进行一定程度上的转换或存储，以使用上述提及的 scikit-learn 提供的评估工具

## 3. 实验记录

In [29]:
# Step 0: import module

from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import os
import random
import datetime

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelBinarizer

from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import gensim
import tensorflow as tf

from IPython.display import display
%matplotlib inline

print("import modules successfully")

import modules successfully


In [2]:
%%time

# Step 1: preprocess the data

paths = {}
paths['dir.dataroot'] =  os.path.join(os.getcwd(), '..', 'data')
paths['dir.train'] = os.path.join(paths['dir.dataroot'], 'trialdata', 'train')
paths['dir.test'] = os.path.join(paths['dir.dataroot'], 'trialdata', 'test')
        
preprocessedFlag = os.path.join(paths['dir.dataroot'], 'preprocessed')
if not os.path.isfile(preprocessedFlag):
    for tpart in ['train', 'test']:
        dirpath = paths['dir.{}'.format(tpart)]
        for cls in os.listdir(dirpath):
            clspath = os.path.join(dirpath, cls)
            files = os.listdir(clspath)
            for f in files:
                fpath = os.path.join(clspath, f)
                os.system('mv {} {}.old'.format(fpath, fpath))
                os.system('perl {} {}.old > {}'.format(os.path.join(paths['dir.dataroot'], 'newfil.pl'), fpath, fpath))
                os.system('rm {}.old'.format(fpath))
    os.system('touch {}'.format(preprocessedFlag))
                
print("file preporcessing succefully")
            
stopwordlist = []
with open(os.path.join(paths['dir.dataroot'], 'stoplist-web.txt'), 'r') as readf:
    stopwordlist = readf.read()
    stopwordlist = stopwordlist.split('\n')
            
print("read stop word list successfully")
        
print("Step 1 Succeed")

file preporcessing succefully
read stop word list successfully
Step 1 Succeed
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.69 ms


### 3.1 BOW+TF-IDF 表示法

In [6]:
%%time

# Step 2: read data and save it in data['nearRaw.train'] 和 data['nearRaw.test']
modelChoice = 'TFIDF'

data = {}
data['nearRaw.train'] = {'content':[], 'class':[]}
data['nearRaw.test'] = {'content':[], 'class':[]}

for tpart in ['train', 'test']:
    dirpath = paths['dir.{}'.format(tpart)]
    for (ind, cls) in enumerate(os.listdir(dirpath)):
        clspath = os.path.join(dirpath, cls)
        files = os.listdir(clspath)
        for f in files:
            fpath = os.path.join(clspath, f)
            with open(fpath, 'r') as readf:
                data['nearRaw.{}'.format(tpart)]['content'].append(readf.read())
                data['nearRaw.{}'.format(tpart)]['class'].append(cls)
    tmp = data['nearRaw.{}'.format(tpart)]
    ind = (random.sample(range(len(tmp['class'])), 1))[0]
    print("sample(transformed) from {}[{}]:\n[content]\n {}\n[class]\n{}".format(
            tpart, ind, tmp['content'][ind], tmp['class'][ind]
        )
    )
    print() 
    
print("Step 2 Succeed")

sample(transformed) from train[2279]:
[content]
  from ed cwis unomaha edu ed stastny subject the otis project ftp sites for original art and images keywords mr owl how many licks organization university of nebraska at omaha lines two two seven the otis project nine three the operative term is stimulate this file last updated four two one nine three what is otis otis is here for the purpose of distributing original artwork and photographs over the network for public perusal scrutiny and distribution digital immortality the basic idea behind digital immortality is that computer networks are here to stay and that anything interesting you deposit on them will be around near forever the gifs and jpgs of today will be the artifacts of a digital future perhaps they ll be put in different formats perhaps only surviving on backup tapes but they ll be there and someone will dig them up if that doesn t interest you otis also offers a forum for critique and exhibition of your works a virtual art 

#### 3.1.1 普通 VSM 表示模型

<center>参数列表</center>

参数 |  取值
--------|-----------
max_df | 0.9
min_df | 0.01
max_features | 784
analyzer | 'word'
stop_words | stopwordlist（来自文件）

In [7]:
%%time

# Step 3: TfidfVectorizer.fit + TfidfVectorizer.transform + save in Pandas.DataFrame
# A. 使用 sklearn.feature_extraction.text.TfidfVectorizer.fit 拟合训练数据，建立 BOW+TF-IDF 
# B. 使用 sklearn.feature_extraction.text.TfidfVectorizer.transform 将data['nearRaw.train']中的 stringContent 和 data['nearRaw.test']中的 stringContent 进行处理，
# 将 BOW+TF-IDF 表示结果输出到 data['matrix.train'] 与 data['matrix.test'] 中，供后续学习和训练使用
# 
# C. 将 data['matrix.train'] 与 data['matrix.test'] 转换成 Pandas.DataFrame 格式，保存到 df['train'] 和 df['test'] 中（df 为字典格式：String -> DataFrame）

from sklearn.feature_extraction.text import TfidfVectorizer

## Substep A: vectorization + TF-IDF calculation
vecsize = 784
vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.01, max_features=vecsize, analyzer='word', stop_words=stopwordlist)
vectorizer.fit(data['nearRaw.train']['content'])

print("Substep A finished.")
print("--------------------------------------------------")

## Substep B: Transformation

for tpart in ['train', 'test']:
    data['matrix.{}'.format(tpart)] = vectorizer.transform(data['nearRaw.{}'.format(tpart)]['content'])
    ind = (random.sample(range(data['matrix.{}'.format(tpart)].shape[0]), 1))[0]
    print("sample for matrix.{}".format(tpart))
    print("from ind: {}".format(ind))
    print(data['matrix.{}'.format(tpart)][ind])
    print() 
    
print("Substep B finished.")
print("--------------------------------------------------")

# Substep C: integrate data into DataFrame format

csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')
if not os.path.isdir(csvpath_root):
    os.mkdir(csvpath_root)

df = {}
for tpart in ['train', 'test']:
    datadict = {}
    datadict['class'] = data['nearRaw.{}'.format(tpart)]['class']
    for col in range(data['matrix.{}'.format(tpart)].shape[1]):
        datadict[col]= [i[0] for i in data['matrix.{}'.format(tpart)].getcol(col).toarray()]
#         datadict[str(col)]= [i[0] for i in data['matrix.{}'.format(tpart)].getcol(col).toarray()]

    df[tpart] = pd.DataFrame(data=datadict)
    print("See df[{}]".format(tpart))
    display(df[tpart])
    print("\n\n\n")
    # write data in DataFrame into CSV
    csvpath = os.path.join(csvpath_root, "{}-{}.csv".format(tpart, modelChoice))
    df[tpart].to_csv(csvpath, columns=df[tpart].columns)

print("Substep C finished.")
print("--------------------------------------------------")

print("Step 3 Succeed.")

# 繁琐点：研究如何把 CSR 矩阵中的数据规整好放到 DataFrame 中，并与 Class 一一对应

Substep A finished.
--------------------------------------------------
sample for matrix.train
from ind: 2385
  (0, 783)	0.285416737335
  (0, 770)	0.185125856098
  (0, 652)	0.219756542021
  (0, 628)	0.112067764994
  (0, 612)	0.11498914104
  (0, 566)	0.180083750663
  (0, 549)	0.170377050063
  (0, 481)	0.0741653714903
  (0, 426)	0.693245185673
  (0, 283)	0.223815276854
  (0, 281)	0.239568315863
  (0, 257)	0.315214396579
  (0, 250)	0.101436255178
  (0, 203)	0.114557050154
  (0, 182)	0.179956076368

sample for matrix.test
from ind: 564
  (0, 783)	0.25037021215
  (0, 781)	0.124487406684
  (0, 775)	0.0561196577336
  (0, 772)	0.141241431695
  (0, 770)	0.0974364719433
  (0, 765)	0.131777847543
  (0, 736)	0.123712296337
  (0, 722)	0.0966474258649
  (0, 711)	0.404116464909
  (0, 708)	0.120212864006
  (0, 691)	0.0996602988254
  (0, 683)	0.103176840655
  (0, 658)	0.316198386017
  (0, 603)	0.0936574438545
  (0, 590)	0.156114988678
  (0, 566)	0.0947826829205
  (0, 549)	0.134510697275
  (0, 523)	0.06

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.151835,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.131008,0.000000,0.140640,0.000000,0.000000,0.000000,0.000000,0.141967,0.116894,soc.religion.christian
1,0.000000,0.000000,0.000000,0.086425,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.017108,0.000000,0.000000,0.000000,0.000000,0.070079,0.000000,0.000000,0.091591,soc.religion.christian
2,0.000000,0.000000,0.097996,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.042926,0.000000,0.184327,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,soc.religion.christian
3,0.073392,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.031662,0.000000,0.067981,0.000000,0.000000,0.000000,0.000000,0.000000,0.113006,soc.religion.christian
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.023204,0.061024,0.000000,0.000000,0.000000,0.000000,0.000000,0.050289,0.041408,soc.religion.christian
5,0.058849,0.000000,0.000000,0.000000,0.070452,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135919,soc.religion.christian
6,0.000000,0.000000,0.000000,0.000000,0.056674,0.000000,0.0,0.000000,0.000000,0.043449,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135911,0.000000,0.000000,soc.religion.christian
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.063593,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.170227,soc.religion.christian
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.060522,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.216009,soc.religion.christian
9,0.000000,0.000000,0.000000,0.246793,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.048853,0.128482,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,soc.religion.christian






See df[test]


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.072939,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.031467,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,soc.religion.christian
1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.078079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,soc.religion.christian
2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.077998,0.103377,0.000000,0.000000,0.000000,soc.religion.christian
3,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.050030,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044641,soc.religion.christian
4,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.210194,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081678,0.000000,soc.religion.christian
5,0.000000,0.0,0.000000,0.000000,0.000000,0.053324,0.0,0.000000,0.000000,0.000000,...,0.019539,0.000000,0.125853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,soc.religion.christian
6,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.092240,0.000000,0.000000,0.080370,soc.religion.christian
7,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.031390,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068031,0.028008,soc.religion.christian
8,0.000000,0.0,0.000000,0.072953,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.200846,0.295775,0.000000,0.000000,0.051542,soc.religion.christian
9,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.059443,...,0.000000,0.000000,0.000000,0.058781,0.000000,0.114455,0.000000,0.000000,0.124657,soc.religion.christian






Substep C finished.
--------------------------------------------------
Step 3 Succeed.
CPU times: user 5.89 s, sys: 96 ms, total: 5.98 s
Wall time: 5.88 s


In [8]:
%%time

# if wanna read data from CSV file

df = {}

for tpart in ['train', 'test']:
    csvpath = os.path.join(paths['dir.dataroot'], 'data_CSV', '{}-{}.csv'.format(tpart, modelChoice))
    if os.path.exists(csvpath):
        df[tpart] = pd.DataFrame.from_csv(csvpath)
        df[tpart] = df[tpart].sample(frac=1)
        df[tpart].reset_index(drop=True, inplace=True)
        print("read {} successfully".format(tpart))
        display(df[tpart])


read train successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.088296,rec.motorcycles
1,0.000000,0.000000,0.270996,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.118705,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052959,rec.motorcycles
2,0.000000,0.000000,0.000000,0.106788,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.126834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037723,alt.atheism
3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.088524,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.513421,rec.motorcycles
4,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.117773,0.000000,0.000000,0.000000,0.000000,0.099905,comp.graphics
5,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.119460,rec.autos
6,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.095337,...,0.089627,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,rec.motorcycles
7,0.039368,0.000000,0.000000,0.000000,0.04713,0.000000,0.000000,0.000000,0.0,0.036132,...,0.067936,0.000000,0.000000,0.035729,0.000000,0.000000,0.075349,0.000000,0.015154,alt.atheism
8,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285417,comp.graphics
9,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.080364,0.000000,0.000000,0.169062,0.000000,0.000000,0.000000,0.000000,0.000000,comp.graphics


read test successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.024113,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052260,0.000000,alt.atheism
1,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.057582,0.0,0.000000,0.000000,...,0.061000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.199572,rec.autos
2,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052024,soc.religion.christian
3,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039734,rec.autos
4,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.057348,0.000000,0.000000,0.120644,0.000000,0.000000,0.000000,0.000000,0.255850,rec.motorcycles
5,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.015311,0.000000,0.000000,0.032210,0.000000,0.000000,0.000000,0.000000,0.068308,soc.religion.christian
6,0.000000,0.0,0.000000,0.0,0.000000,0.109173,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086700,0.000000,alt.atheism
7,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.089744,soc.religion.christian
8,0.099696,0.0,0.196379,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076754,soc.religion.christian
9,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.105101,0.000000,0.000000,0.000000,0.000000,0.267465,rec.autos


CPU times: user 632 ms, sys: 0 ns, total: 632 ms
Wall time: 667 ms


In [10]:
# %%time

# # Step 4: One-hot representation for labels

# csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')

# lb = LabelBinarizer()
# lb.fit(df['train']['class'])

# df_new = {}
# for tpart in ['train', 'test']:
#     labels = lb.transform(df[tpart]['class'])
#     labelsDf = pd.DataFrame(labels, columns=["class-{}".format(i) for i in range(len(lb.classes_))])
#     df_new[tpart] = {}
#     df_new[tpart]['y'] = labelsDf
#     df_new[tpart]['x'] = df[tpart].drop('class', axis=1)
#     df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
#     #save in CSV
#     for subpart in ['x', 'y', 'all']:
#         csvpath = os.path.join(csvpath_root, "{}-cleanLabels-{}-{}.csv".format(tpart, subpart, modelChoice))
#         df_new[tpart][subpart].to_csv(csvpath)
    
# print("label cleaning succussfully")

label cleaning succussfully
CPU times: user 2.05 s, sys: 76 ms, total: 2.12 s
Wall time: 2.12 s


In [24]:
# Step 5: Training

In [11]:
%%time

# Step 5.1.1: SVM

# if 'TFIDF' == modelChoice:

#train
X_train = df['train'].drop('class', axis=1)
y_train = df['train']['class']
#test
X_test = df['test'].drop('class', axis=1)
y_test_true = df['test']['class']

# else:
#     #train
#     X_train = df_new['train']['x']
#     y_train = df_new['train']['y']
#     #test
#     X_test = df_new['test']['x']
#     y_test_true = df_new['test']['y']

clf = LinearSVC()
clf.fit(X_train, y_train)

print("Step 4 finished")

Step 4 finished
CPU times: user 68 ms, sys: 16 ms, total: 84 ms
Wall time: 83 ms


In [12]:
%%time

# Step 5.1.2: Test
y_test_pred = clf.predict(X_test)
print(accuracy_score(y_test_true, y_test_pred))
print(f1_score(y_test_true, y_test_pred, average='macro'))
print(f1_score(y_test_true, y_test_pred, average='micro'))

# print f1_score(y_test_true, tensorPredCls, average='micro')

0.861578947368
0.859091299728
0.861578947368
CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 19 ms


In [13]:
%%time

temps = ""
with open(os.path.join(paths['dir.dataroot'], 'stoplist-baseTFIDF.txt'), 'w') as stoplistfile:
    for w in vectorizer.stop_words_:
        temps += "{} ".format(w)
    stoplistfile.write(temps)
    
print("Output stoplist successfully.")

Output stoplist successfully.
CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 46.3 ms


#### 3.1.2 基于 BOW+TFIDF 的 LSA 表示

<center>参数列表</center>

参数 |  取值
--------|-----------
k(n_components) | 200
n_iter | 10
random_state | 19

In [14]:
%%time

# Step 4: LSA based on BOW+TFIDF
# 注意：sklearn.decomposition.TruncatedSVD 的 fit() 与 transform() 方法接受的参数虽然说要求是稀疏矩阵（sparse matrix），但只要求输入

from sklearn.decomposition import TruncatedSVD

modelChoice = 'LSA'

csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')

svd = TruncatedSVD(n_components=200, n_iter=10, random_state=19)
svd.fit(df['train'].drop('class', axis=1))

df_new = {}
for tpart in ['train', 'test']:
    df_new[tpart] = {}
    datadict = {}
    X_LSA = svd.transform(df[tpart].drop('class', axis=1))
    for col in range(X_LSA.shape[1]):
        datadict[col] = X_LSA[:, col]
    df_new[tpart]['y'] = df[tpart]['class']
    df_new[tpart]['x'] = pd.DataFrame(data=datadict)
    df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
    print('Finish {} for data: {}'.format(modelChoice, tpart))
    display(df_new[tpart]['all'])
    print("\n\n\n")
    #save in CSV
    for subpart in ['x', 'y', 'all']:
        csvpath = os.path.join(csvpath_root, "{}-cleanLabels-{}-{}.csv".format(tpart, subpart, modelChoice))
        df_new[tpart][subpart].to_csv(csvpath)

Finish LSA for data: train


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,class
0,0.271848,0.086854,0.044718,-0.184021,-0.104875,-0.061883,-0.023094,-0.019757,0.072721,-0.174748,...,0.020996,-0.057589,-0.023206,0.044620,0.033468,-0.042031,-0.056432,-0.053197,0.022025,rec.motorcycles
1,0.354713,0.024572,-0.053660,0.027571,-0.112272,0.008970,-0.117263,0.035975,0.068295,-0.064175,...,0.040832,-0.026654,0.012075,0.050080,-0.090460,-0.027934,-0.044482,0.023124,-0.080695,rec.motorcycles
2,0.336806,0.131161,-0.080951,0.061080,0.033372,0.182860,0.146511,0.024233,-0.036709,0.011195,...,0.016687,0.011382,0.001840,-0.011631,0.018478,-0.000071,-0.004590,-0.035387,-0.019772,alt.atheism
3,0.669176,-0.222391,0.154325,-0.113312,-0.008736,0.042402,-0.122434,0.090178,0.086653,-0.010587,...,-0.020793,-0.010716,0.008322,-0.000535,0.042258,-0.040557,-0.000607,-0.007880,-0.022110,rec.motorcycles
4,0.242416,-0.017904,-0.132323,0.110422,0.117907,-0.096068,-0.184019,-0.017587,-0.052250,-0.054797,...,-0.012923,-0.014938,-0.019100,0.031885,-0.027216,-0.009254,0.005577,-0.005521,-0.059192,comp.graphics
5,0.265944,0.053592,-0.089929,-0.205034,-0.135492,0.045844,-0.182254,0.044461,0.185399,0.072694,...,-0.011158,0.000387,0.002401,0.018223,0.032595,-0.072950,-0.046289,0.011751,0.000851,rec.autos
6,0.245897,0.041406,-0.037822,-0.135230,-0.105791,-0.113244,0.051711,0.029380,0.099952,-0.237700,...,0.006944,-0.036590,0.077661,0.012173,0.072640,0.055367,0.008979,0.036428,0.081808,rec.motorcycles
7,0.232699,0.244133,-0.128749,-0.213448,0.189454,0.225212,0.036346,-0.033897,-0.228594,-0.196528,...,-0.034541,0.023703,0.021700,0.006158,-0.039249,0.021697,0.065730,0.032391,0.020442,alt.atheism
8,0.356918,-0.174550,0.037398,0.050916,0.139657,0.043051,-0.035772,-0.085151,0.107688,-0.025660,...,0.061714,0.075787,0.045739,0.027133,-0.002291,-0.013261,-0.026346,-0.009649,0.010169,comp.graphics
9,0.259889,-0.031931,-0.118164,0.042314,-0.168207,0.100546,-0.009619,-0.098718,0.044267,0.007701,...,0.041649,-0.061747,0.029451,-0.027174,-0.066717,-0.023137,-0.021732,0.009132,-0.039401,comp.graphics






Finish LSA for data: test


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,class
0,0.114564,0.164067,-0.024724,-0.030537,0.090902,-0.009459,0.014457,0.019973,-0.052380,-0.020850,...,0.062908,-0.020304,0.072522,0.070766,-0.021639,0.050329,0.061819,0.043197,0.000102,alt.atheism
1,0.519665,-0.021193,0.011763,-0.081025,-0.016817,-0.088999,0.137517,-0.203228,-0.036433,0.073082,...,0.004456,0.041031,-0.002497,0.017334,-0.017513,-0.022711,0.006521,0.051933,0.019353,rec.autos
2,0.400768,-0.004849,0.001060,0.138331,-0.053400,0.017185,0.028488,-0.087491,-0.006689,-0.127574,...,-0.007918,-0.025594,-0.012875,-0.006147,0.002793,-0.037490,0.013872,-0.035135,0.029218,soc.religion.christian
3,0.494432,-0.128792,-0.075977,0.158679,-0.160794,0.070026,0.079511,0.025272,-0.132389,-0.084178,...,-0.042501,-0.037805,-0.003796,-0.010782,0.006794,0.049987,-0.000206,0.030371,0.007290,rec.autos
4,0.440319,-0.061954,-0.023238,-0.122690,0.095754,-0.078904,0.067468,0.146911,0.039260,-0.041248,...,-0.025799,-0.017724,-0.024578,-0.001334,0.025187,0.011958,0.027222,0.035019,-0.032083,rec.motorcycles
5,0.328207,0.181910,0.122593,0.142017,-0.053450,0.042890,0.002027,-0.075838,0.015566,0.030917,...,0.047407,0.066344,-0.022169,0.023059,0.000695,-0.010573,-0.018855,0.054881,0.007070,soc.religion.christian
6,0.272883,0.289963,0.077959,0.143253,0.023714,-0.017974,0.033273,0.009029,0.021902,-0.041701,...,0.011500,0.017489,0.037776,0.001220,-0.016065,0.054652,0.000180,-0.006513,0.015332,alt.atheism
7,0.328090,0.137693,0.135918,0.102294,-0.081237,0.089016,-0.017763,-0.031980,0.150092,0.009629,...,-0.030689,0.006407,-0.001757,-0.077525,0.002236,-0.005762,0.042114,-0.016589,-0.031774,soc.religion.christian
8,0.312789,0.139804,0.065698,0.134523,0.012125,0.002448,-0.053565,0.060769,0.013670,0.032497,...,-0.011592,0.068106,0.020433,-0.011731,0.026763,0.032598,0.016502,-0.018786,0.018138,soc.religion.christian
9,0.511293,-0.162422,0.009341,-0.008767,-0.010792,0.063196,-0.004236,-0.066234,0.123269,-0.025538,...,-0.075358,-0.040903,-0.013458,-0.064500,0.021368,0.049023,-0.035709,0.069203,0.014727,rec.autos






CPU times: user 6.97 s, sys: 3.82 s, total: 10.8 s
Wall time: 2.45 s


In [15]:
%%time

# Step 5: Training

# if 'TFIDF' == modelChoice:
#     #train
#     X_train = df['train'].drop('class', axis=1)
#     y_train = df['train']['class']
#     #test
#     X_test = df['test'].drop('class', axis=1)
#     y_test_true = df['test']['class']
# else:

#train
X_train = df_new['train']['x']
y_train = df_new['train']['y']
#test
X_test = df_new['test']['x']
y_test_true = df_new['test']['y']

clf = LinearSVC()
clf.fit(X_train, y_train)

print("Step 4 finished")

Step 4 finished
CPU times: user 164 ms, sys: 0 ns, total: 164 ms
Wall time: 164 ms


In [16]:
%%time

# Step 6: Evaluate

y_test_pred = clf.predict(X_test)
print(accuracy_score(y_test_true, y_test_pred))
print(f1_score(y_test_true, y_test_pred, average='macro'))
print(f1_score(y_test_true, y_test_pred, average='micro'))

0.867368421053
0.863905609159
0.867368421053
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 15 ms


#### 3.1.3 基于 BOW+TFIDF 的 LDA 表示

<center>参数列表</center>

参数 |  取值
--------|-----------
n_topic | 50
max_iter | 10
random_state | 19

> 注意：实验表明，主题数的选择会影响训练效果。目前测试过 5, 35, 50, 75, 100, 200，似乎以 50 效果最好。因此最后保留 50 的训练结果。

In [29]:
%%time

# Step 4: LDA based on BOW+TFIDF
# 注意：sklearn.decomposition.TruncatedSVD 的 fit() 与 transform() 方法接受的参数虽然说要求是稀疏矩阵（sparse matrix），但只要求输入

from sklearn.decomposition import LatentDirichletAllocation

modelChoice = 'LDA'

csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')

lda = LatentDirichletAllocation(n_topics=50, max_iter=10, random_state=19, learning_method='batch')
lda.fit(df['train'].drop('class', axis=1))

df_new = {}
for tpart in ['train', 'test']:
    df_new[tpart] = {}
    datadict = {}
    X_LDA = lda.transform(df[tpart].drop('class', axis=1))
    for col in range(X_LDA.shape[1]):
        datadict[col] = X_LDA[:, col]
    df_new[tpart]['y'] = df[tpart]['class']
    df_new[tpart]['x'] = pd.DataFrame(data=datadict)
    df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
    print('Finish {} for data: {}'.format(modelChoice, tpart))
    display(df_new[tpart]['all'])
    print("\n\n\n")
    #save in CSV
    for subpart in ['x', 'y', 'all']:
        csvpath = os.path.join(csvpath_root, "{}-cleanLabels-{}-{}.csv".format(tpart, subpart, modelChoice))
        df_new[tpart][subpart].to_csv(csvpath)

Finish LDA for data: train


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,class
0,0.002856,0.002856,0.002856,0.002856,0.002856,0.002856,0.002856,0.002856,0.002856,0.002856,...,0.002856,0.002856,0.002856,0.002856,0.022632,0.002856,0.002856,0.002856,0.002856,rec.motorcycles
1,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,...,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,0.002700,rec.motorcycles
2,0.002795,0.153632,0.002795,0.002795,0.002795,0.002795,0.002795,0.002795,0.002795,0.002795,...,0.002795,0.002795,0.002795,0.002795,0.002795,0.002795,0.132057,0.002795,0.002795,alt.atheism
3,0.002905,0.002905,0.002905,0.002905,0.002905,0.002905,0.002905,0.002905,0.002905,0.002905,...,0.002905,0.002905,0.002905,0.002905,0.019844,0.002905,0.002905,0.002905,0.002905,rec.motorcycles
4,0.003393,0.003393,0.003393,0.003393,0.003393,0.003393,0.003393,0.003393,0.003393,0.003393,...,0.367731,0.003393,0.003393,0.003393,0.003393,0.003393,0.003393,0.003393,0.003393,comp.graphics
5,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,...,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,0.004160,rec.autos
6,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,...,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,0.002825,rec.motorcycles
7,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,...,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,0.002394,alt.atheism
8,0.004751,0.004751,0.004751,0.004751,0.004751,0.004751,0.004751,0.004751,0.004751,0.004751,...,0.004751,0.004751,0.004751,0.004751,0.462776,0.004751,0.004751,0.004751,0.004751,comp.graphics
9,0.003754,0.003754,0.053233,0.003754,0.003754,0.003754,0.003754,0.003754,0.003754,0.003754,...,0.045130,0.003754,0.003754,0.003754,0.003754,0.003754,0.003754,0.003754,0.003754,comp.graphics






Finish LDA for data: test


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,class
0,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,...,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,0.003941,alt.atheism
1,0.002108,0.002108,0.002108,0.002108,0.002108,0.002108,0.002108,0.002108,0.002108,0.002108,...,0.002108,0.002108,0.002108,0.002108,0.002108,0.002108,0.008482,0.002108,0.002108,rec.autos
2,0.002848,0.002848,0.002848,0.002848,0.002848,0.002848,0.002848,0.002848,0.002848,0.002848,...,0.002848,0.002848,0.002848,0.002848,0.254074,0.002848,0.002848,0.002848,0.002848,soc.religion.christian
3,0.003116,0.003116,0.003116,0.003116,0.003116,0.003116,0.003116,0.003116,0.003116,0.003116,...,0.003116,0.003116,0.003116,0.106555,0.024955,0.003116,0.003116,0.003116,0.003116,rec.autos
4,0.002868,0.002868,0.002868,0.002868,0.002868,0.002868,0.002868,0.002868,0.002868,0.002868,...,0.002868,0.002868,0.002868,0.115058,0.002868,0.002868,0.002868,0.002868,0.002868,rec.motorcycles
5,0.002355,0.002355,0.002355,0.002355,0.002355,0.002355,0.002355,0.002355,0.002355,0.002355,...,0.002355,0.002355,0.002355,0.002355,0.002355,0.002355,0.018169,0.002355,0.002355,soc.religion.christian
6,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,...,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,0.002652,alt.atheism
7,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,...,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,0.003309,soc.religion.christian
8,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,...,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,0.003099,soc.religion.christian
9,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,...,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,0.002785,rec.autos






CPU times: user 19.8 s, sys: 58.2 s, total: 1min 17s
Wall time: 10.5 s


In [31]:
%%time

# Step 5: Training

# if 'TFIDF' == modelChoice:
#     #train
#     X_train = df['train'].drop('class', axis=1)
#     y_train = df['train']['class']
#     #test
#     X_test = df['test'].drop('class', axis=1)
#     y_test_true = df['test']['class']
# else:

#train
X_train = df_new['train']['x']
y_train = df_new['train']['y']
#test
X_test = df_new['test']['x']
y_test_true = df_new['test']['y']
    
clf = LinearSVC()
clf.fit(X_train, y_train)

print("Step 4 finished")

Step 4 finished
CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 97.7 ms


In [32]:
%%time

# Step 6: Evaluate

y_test_pred = clf.predict(X_test)
print(accuracy_score(y_test_true, y_test_pred))
print(f1_score(y_test_true, y_test_pred, average='macro'))
print(f1_score(y_test_true, y_test_pred, average='micro'))

0.593157894737
0.573493123176
0.593157894737
CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 13.5 ms


### 3.2 Word2Vec 表示法

In [3]:
paths['dir.modelroot'] = os.path.join(paths['dir.dataroot'], '..', 'models')
for modeltool in ['gensim', 'TensorFlow']:
    for embedsource in ['text8', 'corpus']:
        dname = os.path.join(paths['dir.modelroot'], '{}.{}'.format(modeltool, embedsource))
        if not os.path.isdir(dname):
            os.mkdir(dname)
        paths['dir.{}.{}'.format(modeltool, embedsource)] = dname

#### 3.2.1 gensim 训练

In [104]:
# Step 0: Import modules

modelFrom = 'gensim'

print("Step 0 succeed.")

Step 0 succeed.


##### 3.2.1.1 基于 text8 建模

In [6]:
%%time

# Step 1: preprocess the data

# import stoplist
stopwords = ""

pathtemp_TFIDF = os.path.join(paths['dir.dataroot'], 'stoplist-baseTFIDF.txt')
with open(pathtemp_TFIDF, 'r') as stoplistfile:
    stopwords = stoplistfile.read()
stopwords = stopwords.split()

pathtemp_web = os.path.join(paths['dir.dataroot'], 'stoplist-web.txt')
with open(pathtemp_web, 'r') as stoplistfile2:
    stopwords2 = stoplistfile2.read()
    stopwords2 = stopwords2.split('\n')
    stopwords = set(stopwords)
    stopwords = list(stopwords.union(set(stopwords)))
    
print("Read stop words successfully.")

Read stop words successfully.
CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 23.2 ms


In [7]:
%%time
   
trimmer = lambda word, count, min_count: gensim.utils.RULE_DISCARD if word in stopwords else gensim.utils.RULE_KEEP

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [8]:
%%time

embedFrom = 'text8'
text8fname = os.path.join(paths['dir.dataroot'], 'text8')
sentences = gensim.models.word2vec.Text8Corpus(text8fname)

vecsize = 784
model = gensim.models.word2vec.Word2Vec(iter=15, size=vecsize, sg=1, window=5, workers=12)
model.build_vocab(sentences=sentences, trim_rule=trimmer)
model.train(sentences=sentences)

CPU times: user 2h 13min 36s, sys: 8.62 s, total: 2h 13min 45s
Wall time: 31min 56s


In [15]:
%%time

# save model

calander = datetime.date.today().timetuple()

modelpath = os.path.join(
    paths['dir.{}.{}'.format(modelFrom, embedFrom)],
    '{}.{}.{}{}{}'.format(modelFrom, embedFrom, calander.tm_year, calander.tm_mon, calander.tm_mday)
)
model.save(modelpath)

print("save model finished")

save model finished
CPU times: user 2.58 s, sys: 228 ms, total: 2.81 s
Wall time: 12.5 s


In [17]:
%%time

# Step 2: read data and save it in data['vec.train'] 和 data['vec.test']

data = {}
data['vec.train'] = {'w2v.mean':[], 'class':[]}
data['vec.test'] = {'w2v.mean':[], 'class':[]}

for tpart in ['train', 'test']:
    dirpath = paths['dir.{}'.format(tpart)]
    for (ind, cls) in enumerate(os.listdir(dirpath)):
        clspath = os.path.join(dirpath, cls)
        files = os.listdir(clspath)
        for f in files:
            fpath = os.path.join(clspath, f)
            with open(fpath, 'r') as readf:
                tokens = [token for token in readf.read().split()] # if token not in stopwords]#readf.read().split()#
                # Word2Vec representation
                # begin
                vec = np.array([0.0 for i in range(vecsize)])
                expectationVal = np.array([0.0 for i in range(vecsize)])
                countvec = 0
                for token in tokens:
                    try:
                        vec += model[token]
                        countvec += 1
                    except:
                        vec += expectationVal
                vec = vec / float(countvec)#float(len(tokens))
                 # end
            data['vec.{}'.format(tpart)]['w2v.mean'].append(vec)
            data['vec.{}'.format(tpart)]['class'].append(cls)

    tmp = data['vec.{}'.format(tpart)]
    ind = (random.sample(range(len(tmp['class'])), 1))[0]
    print("sample(transformed) from {}[{}]:\n[corpus]\n {}\n[class]\n{}".format(
            tpart, ind, tmp['w2v.mean'][ind], tmp['class'][ind]
        )
    )
    print()
    
print("Step 2 Succeed")

sample(transformed) from train[1294]:
[corpus]
 [  1.42176096e-01   1.62430700e-01  -8.79922659e-02   5.35091242e-02
   8.59380437e-02   2.62035794e-04   7.22310296e-02   2.61135434e-02
  -2.16664064e-02   5.44147822e-02   1.06476713e-01  -5.75761132e-02
  -1.10888660e-01  -2.65366570e-02  -1.12344211e-01   7.02185809e-02
   2.53348814e-02   1.30706466e-01  -1.04799571e-01   4.39703308e-02
  -9.66833411e-03   1.64926087e-02   7.21110973e-02   4.64726405e-02
   1.42359131e-01   2.71736607e-02  -1.44038068e-03  -3.01274558e-02
  -1.66951142e-01  -1.89049134e-01  -1.28849644e-01   1.76089907e-01
   7.72865493e-02  -3.66129199e-02  -1.38183594e-02   2.13841639e-02
   1.38888888e-02   1.29469739e-01   1.23148409e-01  -1.73164004e-01
   2.79120405e-02  -7.15156136e-02  -1.03259663e-02   6.85949264e-03
   5.14812296e-03  -1.38993850e-01  -2.75248113e-02   5.88013921e-02
  -1.28485937e-02   2.55639031e-02  -5.69223881e-03   6.49880191e-03
   1.56568261e-02  -3.84757851e-02  -4.55514945e-02  -5

sample(transformed) from test[1009]:
[corpus]
 [  1.57098071e-01   1.53154620e-01  -9.61496837e-02   7.66679577e-02
   9.88287428e-02   2.56884951e-02   4.97653553e-02   1.61335431e-02
  -4.28836131e-02   8.93512531e-02   8.97204755e-02  -4.42556309e-02
  -1.20880319e-01  -3.39067199e-02  -1.21989825e-01   6.05836994e-02
   2.43342272e-02   1.07924186e-01  -7.54850704e-02   2.36673727e-02
  -1.31180015e-02   3.76038207e-02   5.74989647e-02   4.92838261e-02
   1.01037374e-01   3.15683201e-02   2.09713250e-02  -1.74226246e-02
  -1.54403559e-01  -1.89296066e-01  -1.21736109e-01   1.71895260e-01
   5.00430703e-02  -5.84864492e-02  -1.67442754e-02   1.83596857e-02
   2.22139320e-02   1.44453748e-01   1.37844339e-01  -1.70359024e-01
   1.47091010e-02  -7.39391100e-02  -1.53880306e-02   6.87595693e-03
   1.30805930e-02  -1.23579607e-01  -2.07829833e-02   3.87572804e-02
  -2.03224753e-02   2.31136564e-02  -3.07712576e-02   1.17793162e-02
   9.60251690e-03  -2.09789647e-02  -2.84358605e-02  -3.

In [19]:
%%time

# Step 3: Save in Pandas.DataFrame
#
# 将 data['matrix.train'] 与 data['matrix.test'] 转换成 Pandas.DataFrame 格式，保存到 df['train'] 和 df['test'] 中（df 为字典格式：String -> DataFrame）

df = {}
csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')
for tpart in ['train', 'test']:
    datadict = {}
    datadict['class'] = data['vec.{}'.format(tpart)]['class']
    datavec = np.array(data['vec.{}'.format(tpart)]['w2v.mean'])
    for col in range(vecsize):
        datadict[col]= datavec[:, col]

    df[tpart] = pd.DataFrame(data=datadict)
    print("See df[{}]".format(tpart))
    display(df[tpart])
    print("\n\n\n")
    # write data in DataFrame into CSV
    csvpath = os.path.join(csvpath_root, '{}-w2v-{}-{}.csv'.format(tpart, embedFrom, modelFrom))
    df[tpart].to_csv(csvpath, columns=df[tpart].columns)
    
print("Step 3 Succeed.")

# 繁琐点：研究如何把 CSR 矩阵中的数据规整好放到 DataFrame 中，并与 Class 一一对应

See df[train]


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.147760,0.191871,-0.123469,0.066009,0.132763,-0.006644,0.044266,0.059715,-0.024833,0.091452,...,-0.099979,-0.074620,0.042913,0.040350,0.134450,0.018293,0.017293,-0.075392,0.078175,soc.religion.christian
1,0.178941,0.187470,-0.079834,0.107205,0.166234,0.019693,0.040066,-0.019030,-0.072647,0.133745,...,-0.118303,-0.016686,0.043755,0.055443,0.155301,0.016082,0.033731,-0.099639,0.073829,soc.religion.christian
2,0.202831,0.170065,-0.097257,0.128243,0.140756,0.013609,0.016742,0.029650,-0.049912,0.088734,...,-0.092249,-0.034035,0.056689,0.049364,0.157539,0.037136,0.029556,-0.080967,0.077829,soc.religion.christian
3,0.208845,0.245270,-0.103284,0.091380,0.157237,0.048984,0.036000,0.005156,-0.052962,0.120857,...,-0.105384,-0.043563,0.055596,0.056120,0.121999,0.023624,0.019472,-0.115223,0.068031,soc.religion.christian
4,0.185805,0.200608,-0.090226,0.083986,0.157810,0.045228,0.077690,0.000514,-0.020258,0.106118,...,-0.108957,-0.053231,0.061687,0.034940,0.137244,0.024679,0.039585,-0.137030,0.093189,soc.religion.christian
5,0.105656,0.135173,-0.043216,0.047694,0.105946,-0.001172,0.115349,0.059904,-0.023608,0.039884,...,-0.107429,-0.093504,0.029772,0.049526,0.099876,0.017826,0.096981,-0.123978,0.081878,soc.religion.christian
6,0.178755,0.182410,-0.090002,0.091815,0.152302,0.048337,0.032308,0.022511,-0.033339,0.115746,...,-0.089736,-0.051238,0.057909,0.061582,0.130873,0.013066,0.029940,-0.105021,0.095332,soc.religion.christian
7,0.090505,0.109543,-0.060146,0.057255,0.110668,-0.001868,0.093157,0.022505,-0.005506,0.042767,...,-0.079171,-0.063464,0.037490,0.063633,0.130327,0.005175,0.059961,-0.070104,0.083359,soc.religion.christian
8,0.111884,0.144636,-0.063390,0.072714,0.108716,0.017135,0.085774,0.080821,-0.028071,0.031066,...,-0.089500,-0.075519,0.033466,0.047627,0.125210,0.003856,0.049750,-0.059231,0.058041,soc.religion.christian
9,0.135486,0.172539,-0.071040,0.081852,0.130687,0.036760,0.045177,0.042558,-0.025867,0.066756,...,-0.093644,-0.043220,0.054607,0.059081,0.128636,0.029543,0.027708,-0.092555,0.107019,soc.religion.christian






See df[test]


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.181281,0.228398,-0.075890,0.098953,0.124880,0.040266,0.058507,0.023693,-0.042918,0.126799,...,-0.098906,-0.036052,0.050036,0.045954,0.131378,0.003953,0.044525,-0.122465,0.071459,soc.religion.christian
1,0.166943,0.141915,-0.058615,0.072855,0.123155,0.022023,0.037875,-0.002635,-0.073110,0.098993,...,-0.061153,-0.044855,0.026496,0.037081,0.146074,-0.016054,0.070424,-0.099942,0.069886,soc.religion.christian
2,0.146988,0.125112,-0.043065,0.094749,0.106156,0.000753,0.046489,0.014582,-0.064784,0.105506,...,-0.087145,-0.038346,0.045473,0.038739,0.150991,0.013829,0.041216,-0.082288,0.088481,soc.religion.christian
3,0.131239,0.177776,-0.054687,0.067915,0.103457,0.034679,0.087555,0.032761,-0.032261,0.086895,...,-0.110234,-0.073667,-0.015017,0.069666,0.147622,-0.010669,0.076071,-0.098396,0.066565,soc.religion.christian
4,0.160831,0.223853,-0.106787,0.097534,0.147625,0.072289,0.030739,0.009655,-0.036697,0.121479,...,-0.113563,-0.052026,0.046215,0.045257,0.130601,0.008114,0.055839,-0.133417,0.061739,soc.religion.christian
5,0.212691,0.201205,-0.110092,0.121360,0.171044,0.069911,0.012311,-0.013675,-0.047791,0.135481,...,-0.125282,-0.044721,0.051155,0.076026,0.135150,0.029299,0.025845,-0.101702,0.086462,soc.religion.christian
6,0.171318,0.173036,-0.055780,0.084935,0.139292,0.023904,0.041544,0.033871,-0.033655,0.101461,...,-0.114612,-0.062470,0.040511,0.047051,0.128280,-0.008847,0.051515,-0.099396,0.076984,soc.religion.christian
7,0.179166,0.204333,-0.074029,0.094361,0.138624,0.024829,0.047687,0.001240,-0.046740,0.105575,...,-0.090121,-0.025891,0.040806,0.040483,0.132452,0.001379,0.031532,-0.087991,0.074717,soc.religion.christian
8,0.194129,0.174792,-0.083872,0.109072,0.140045,0.016814,0.043043,0.039243,-0.058386,0.111187,...,-0.101575,-0.057712,0.077170,0.042844,0.162748,0.002867,0.039176,-0.076533,0.077580,soc.religion.christian
9,0.135488,0.154116,-0.044559,0.073116,0.123987,0.040695,0.047806,0.016156,-0.045030,0.072989,...,-0.100252,-0.058510,0.050253,0.046008,0.146536,0.009696,0.044494,-0.084077,0.061215,soc.religion.christian






Step 3 Succeed.
CPU times: user 2.07 s, sys: 56 ms, total: 2.12 s
Wall time: 2.17 s


In [107]:
%%time

# if wanna read data from CSV file

df = {}

for tpart in ['train', 'test']:
    csvpath = os.path.join(
        csvpath_root, '{}-w2v-{}-{}.csv'.format(
            tpart, embedFrom, modelFrom
        )
    )
    if os.path.exists(csvpath):
        df[tpart] = pd.DataFrame.from_csv(csvpath)
        df[tpart] = df[tpart].sample(frac=1)
        df[tpart].reset_index(drop=True, inplace=True)
        print("read {} successfully".format(tpart))
        display(df[tpart])

read train successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.177065,0.220111,-0.144173,0.102411,0.097858,0.019923,0.061870,0.113750,0.052380,0.011849,...,-0.119355,-0.109509,0.017502,0.072398,0.094370,0.002441,0.068058,-0.141721,0.118900,comp.graphics
1,0.150134,0.171049,-0.060894,0.035372,0.127200,0.028285,0.075184,0.016150,-0.004238,0.043623,...,-0.068356,-0.083557,0.029568,0.096670,0.092045,0.050060,0.028716,-0.099589,0.038581,comp.graphics
2,0.112585,0.129564,-0.096344,0.075622,0.157670,0.073118,0.048869,0.016336,-0.020263,0.072829,...,-0.059083,-0.052792,0.033013,0.061560,0.150799,0.016177,0.059237,-0.104495,0.098079,soc.religion.christian
3,0.168117,0.169154,-0.078624,0.084069,0.112711,0.024250,0.064745,0.033847,-0.002577,0.059480,...,-0.086128,-0.097710,0.035511,0.051856,0.113769,-0.002649,0.061280,-0.134222,0.075854,comp.graphics
4,0.141454,0.179205,-0.080936,0.077810,0.106397,0.034038,0.074566,0.020144,0.021634,0.098362,...,-0.105267,-0.063068,0.024644,0.034122,0.119906,-0.002638,0.059990,-0.098128,0.061528,soc.religion.christian
5,0.113995,0.096731,-0.069479,0.079731,0.107044,-0.034205,0.068047,0.042622,-0.001899,0.031603,...,-0.089813,-0.068895,0.031148,0.029334,0.097274,-0.012348,0.045399,-0.068110,0.086690,comp.graphics
6,0.205406,0.228389,-0.157220,0.121508,0.150480,0.032163,0.029472,0.017817,-0.011324,0.138975,...,-0.114809,-0.035353,0.077374,0.071350,0.093895,0.007648,0.033415,-0.093949,0.080868,soc.religion.christian
7,0.091921,0.119454,-0.075911,0.022086,0.119876,0.031191,0.005383,0.010170,-0.056288,0.046024,...,-0.111086,-0.034373,0.022683,0.056666,0.129046,-0.003815,0.075984,-0.101797,0.119218,soc.religion.christian
8,0.136040,0.130605,-0.019988,0.053304,0.095861,0.036624,0.086791,0.022059,-0.032591,0.096640,...,-0.094072,-0.075990,0.031536,0.001119,0.111031,0.004717,0.073187,-0.119843,0.092968,rec.motorcycles
9,0.175618,0.133833,-0.115383,0.085493,0.079358,-0.024408,0.049117,0.039929,-0.013472,0.043383,...,-0.093006,-0.068485,0.006319,0.030944,0.109105,0.016425,0.056066,-0.087943,0.098050,rec.autos


read test successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.055221,0.074701,-0.059779,0.047216,0.091813,-0.038038,0.093739,0.049682,-0.008024,0.054676,...,-0.090123,-0.063039,0.028016,0.049200,0.102679,-0.006638,0.088112,-0.084938,0.119870,soc.religion.christian
1,0.123648,0.142640,-0.050926,0.069142,0.085433,0.033125,0.103106,0.025281,-0.023796,0.034686,...,-0.098824,-0.055011,0.013116,0.027032,0.090893,-0.013875,0.057162,-0.142370,0.086493,rec.motorcycles
2,0.213773,0.273997,-0.092123,0.057986,0.163515,0.074589,0.058103,0.021981,0.015551,0.060478,...,-0.118686,-0.044158,0.036226,0.032314,0.106265,0.045570,0.019126,-0.133571,0.120159,soc.religion.christian
3,0.157166,0.165131,-0.074020,0.041901,0.103741,-0.003904,0.070539,0.010501,0.001023,0.068057,...,-0.061571,-0.086888,0.025970,0.015124,0.097172,-0.011317,0.049156,-0.117782,0.089839,rec.motorcycles
4,0.157254,0.193971,-0.097019,0.025098,0.148397,0.027434,0.037771,0.038489,-0.065917,0.103195,...,-0.081350,-0.064140,0.020080,0.070574,0.075425,-0.009907,0.061753,-0.132262,0.072737,comp.graphics
5,0.094931,0.158852,-0.083298,0.036415,0.061289,0.039578,0.131851,0.079916,-0.036807,0.013662,...,-0.071642,-0.089384,-0.025363,0.029126,0.088830,-0.017244,0.041484,-0.128943,0.082857,rec.motorcycles
6,0.071874,0.081405,-0.021858,0.022914,0.056762,-0.018064,0.104337,0.057867,-0.001098,0.031533,...,-0.097016,-0.073418,-0.008779,0.026716,0.137433,-0.026200,0.063651,-0.092392,0.074638,rec.autos
7,0.042522,0.043513,-0.066467,0.065892,0.002371,-0.048086,0.099314,0.090553,-0.010046,0.003057,...,-0.064708,-0.085967,-0.014625,0.017201,0.129812,-0.008780,0.100066,-0.101729,0.073067,rec.motorcycles
8,0.183396,0.187342,-0.087993,0.083409,0.112866,0.055050,0.091043,0.040141,0.007992,0.079223,...,-0.101577,-0.082538,0.027820,0.054967,0.129546,0.011346,0.042986,-0.125952,0.090539,alt.atheism
9,0.033685,0.095458,-0.030138,0.075265,0.045467,0.012184,0.143523,0.069437,-0.031456,0.016175,...,-0.056311,-0.091113,-0.041184,0.018035,0.124233,-0.034256,0.094540,-0.090154,0.080557,rec.motorcycles


CPU times: user 944 ms, sys: 0 ns, total: 944 ms
Wall time: 944 ms


###### SVM classifier

In [108]:
%%time

# Step 5.1.1: SVM

# if 'TFIDF' == modelChoice:

#train
X_train = df['train'].drop('class', axis=1)
y_train = df['train']['class']
#test
X_test = df['test'].drop('class', axis=1)
y_test_true = df['test']['class']

# else:
#     #train
#     X_train = df_new['train']['x']
#     y_train = df_new['train']['y']
#     #test
#     X_test = df_new['test']['x']
#     y_test_true = df_new['test']['y']

clf = LinearSVC()
clf.fit(X_train, y_train)

print("Step 4 finished")

Step 4 finished
CPU times: user 1.99 s, sys: 0 ns, total: 1.99 s
Wall time: 1.99 s


In [109]:
%%time
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Step 5.1.2: Test
y_test_pred = clf.predict(X_test)
print(accuracy_score(y_test_true, y_test_pred))
print(f1_score(y_test_true, y_test_pred, average='macro'))
print(f1_score(y_test_true, y_test_pred, average='micro'))

0.830526315789
0.827658664987
0.830526315789
CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 21.3 ms


###### DNN classifier

In [110]:
%%time

# Step 4: One-hot representation for labels

csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')

lb = LabelBinarizer()
lb.fit(df['train']['class'])

df_new = {}
for tpart in ['train', 'test']:
    labels = lb.transform(df[tpart]['class'])
    labelsDf = pd.DataFrame(labels, columns=["class-{}".format(i) for i in range(len(lb.classes_))])
    df_new[tpart] = {}
    df_new[tpart]['y'] = labelsDf
    df_new[tpart]['x'] = df[tpart].drop('class', axis=1)
    df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
    #save in CSV
    for subpart in ['x', 'y', 'all']:
        csvpath = os.path.join(csvpath_root, "{}-cleanLabels-{}-{}.csv".format(tpart, subpart, modelFrom))
        df_new[tpart][subpart].to_csv(csvpath)
    
print("label cleaning succussfully")

label cleaning succussfully
CPU times: user 3.64 s, sys: 108 ms, total: 3.75 s
Wall time: 3.75 s


In [111]:
%%time

## Step 5 : Train the classifier

COL_OUTCOME = 'class'
COL_FEATURE = [str(col) for col in list(df['train'].columns) if col != COL_OUTCOME]

cls2num = {cls:ind for (ind, cls) in enumerate(df['train']['class'].unique())}

def my_input_fn(dataset):
    # Save dataset in tf format
    feature_cols = {
        str(col): tf.constant(
            df[dataset][str(col)].values
        )
        for col in COL_FEATURE
    }
    labels = tf.constant([cls2num[labelname] for labelname in df[dataset][COL_OUTCOME].values])
    # Returns the feature columns and labels in tf format
    return feature_cols, labels

feature_columns = [tf.contrib.layers.real_valued_column(column_name=str(col)) for col in COL_FEATURE]
clf = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns, 
    hidden_units=[512], 
    n_classes=len(df['train']['class'].unique())
)

clf.fit(input_fn=lambda: my_input_fn('train'), steps=2000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f152ac0e1d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp26dqmD/model.ckpt.
INFO:tensorflow:loss = 1.61282, step = 1
INFO:tensorflow:global_step/sec: 11.1793
INFO:tensorflow:loss = 1.36607, step = 101
INFO:tensorflow:global_step/sec: 11.0008
INFO:tensorflow:loss = 1.15435, step = 201
INFO:tensorflow:global_step/sec: 10.8384
INFO:tensorflow:loss = 0.987877, step = 301
INFO:tensorflow:global_step/sec: 11.5904
INFO:tensorflow:loss = 0.858602, step = 401
INFO:tensorflow:global_step/sec: 11.781
INFO:tensorflow:loss = 0.762772, step = 501
INFO:tensorflow:global_step/sec: 11.7706
INFO:tensorflow:loss = 0.690341, step = 601
INFO:t

In [112]:
%%time

## Step 6: Evaluate

accuracy_score = clf.evaluate(input_fn=lambda: my_input_fn('test'), steps=df['test'].shape[0])['accuracy']
print("Test Accuracy by TensorFlow: {}".format(accuracy_score))

X_tensor_test, yt = my_input_fn('test')
tensorPredCls = list(clf.predict(input_fn=lambda: my_input_fn('test')))
num2cls = {v:k for (k, v) in cls2num.items()}
tensorPredClsStr = [num2cls[i] for i in tensorPredCls]
y_test_true = df['test']['class']
print('Test Accuracy by Scikit-learn: ', f1_score(y_test_true, tensorPredClsStr, average='micro'))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-13-10:00:32
INFO:tensorflow:Evaluation [1/1900]
INFO:tensorflow:Evaluation [2/1900]
INFO:tensorflow:Evaluation [3/1900]
INFO:tensorflow:Evaluation [4/1900]
INFO:tensorflow:Evaluation [5/1900]
INFO:tensorflow:Evaluation [6/1900]
INFO:tensorflow:Evaluation [7/1900]
INFO:tensorflow:Evaluation [8/1900]
INFO:tensorflow:Evaluation [9/1900]
INFO:tensorflow:Evaluation [10/1900]
INFO:tensorflow:Evaluation [11/1900]
INFO:tensorflow:Evaluation [12/1900]
INFO:tensorflow:Evaluation [13/1900]
INFO:tensorflow:Evaluation [14/1900]
INFO:tensorflow:Evaluation [15/1900]
INFO:tensorflow:Evaluation [16/1900]
INFO:tensorflow:Evaluation [1

INFO:tensorflow:Evaluation [73/1900]
INFO:tensorflow:Evaluation [74/1900]
INFO:tensorflow:Evaluation [75/1900]
INFO:tensorflow:Evaluation [76/1900]
INFO:tensorflow:Evaluation [77/1900]
INFO:tensorflow:Evaluation [78/1900]
INFO:tensorflow:Evaluation [79/1900]
INFO:tensorflow:Evaluation [80/1900]
INFO:tensorflow:Evaluation [81/1900]
INFO:tensorflow:Evaluation [82/1900]
INFO:tensorflow:Evaluation [83/1900]
INFO:tensorflow:Evaluation [84/1900]
INFO:tensorflow:Evaluation [85/1900]
INFO:tensorflow:Evaluation [86/1900]
INFO:tensorflow:Evaluation [87/1900]
INFO:tensorflow:Evaluation [88/1900]
INFO:tensorflow:Evaluation [89/1900]
INFO:tensorflow:Evaluation [90/1900]
INFO:tensorflow:Evaluation [91/1900]
INFO:tensorflow:Evaluation [92/1900]
INFO:tensorflow:Evaluation [93/1900]
INFO:tensorflow:Evaluation [94/1900]
INFO:tensorflow:Evaluation [95/1900]
INFO:tensorflow:Evaluation [96/1900]
INFO:tensorflow:Evaluation [97/1900]
INFO:tensorflow:Evaluation [98/1900]
INFO:tensorflow:Evaluation [99/1900]
I

INFO:tensorflow:Evaluation [290/1900]
INFO:tensorflow:Evaluation [291/1900]
INFO:tensorflow:Evaluation [292/1900]
INFO:tensorflow:Evaluation [293/1900]
INFO:tensorflow:Evaluation [294/1900]
INFO:tensorflow:Evaluation [295/1900]
INFO:tensorflow:Evaluation [296/1900]
INFO:tensorflow:Evaluation [297/1900]
INFO:tensorflow:Evaluation [298/1900]
INFO:tensorflow:Evaluation [299/1900]
INFO:tensorflow:Evaluation [300/1900]
INFO:tensorflow:Evaluation [301/1900]
INFO:tensorflow:Evaluation [302/1900]
INFO:tensorflow:Evaluation [303/1900]
INFO:tensorflow:Evaluation [304/1900]
INFO:tensorflow:Evaluation [305/1900]
INFO:tensorflow:Evaluation [306/1900]
INFO:tensorflow:Evaluation [307/1900]
INFO:tensorflow:Evaluation [308/1900]
INFO:tensorflow:Evaluation [309/1900]
INFO:tensorflow:Evaluation [310/1900]
INFO:tensorflow:Evaluation [311/1900]
INFO:tensorflow:Evaluation [312/1900]
INFO:tensorflow:Evaluation [313/1900]
INFO:tensorflow:Evaluation [314/1900]
INFO:tensorflow:Evaluation [315/1900]
INFO:tensorf

INFO:tensorflow:Evaluation [506/1900]
INFO:tensorflow:Evaluation [507/1900]
INFO:tensorflow:Evaluation [508/1900]
INFO:tensorflow:Evaluation [509/1900]
INFO:tensorflow:Evaluation [510/1900]
INFO:tensorflow:Evaluation [511/1900]
INFO:tensorflow:Evaluation [512/1900]
INFO:tensorflow:Evaluation [513/1900]
INFO:tensorflow:Evaluation [514/1900]
INFO:tensorflow:Evaluation [515/1900]
INFO:tensorflow:Evaluation [516/1900]
INFO:tensorflow:Evaluation [517/1900]
INFO:tensorflow:Evaluation [518/1900]
INFO:tensorflow:Evaluation [519/1900]
INFO:tensorflow:Evaluation [520/1900]
INFO:tensorflow:Evaluation [521/1900]
INFO:tensorflow:Evaluation [522/1900]
INFO:tensorflow:Evaluation [523/1900]
INFO:tensorflow:Evaluation [524/1900]
INFO:tensorflow:Evaluation [525/1900]
INFO:tensorflow:Evaluation [526/1900]
INFO:tensorflow:Evaluation [527/1900]
INFO:tensorflow:Evaluation [528/1900]
INFO:tensorflow:Evaluation [529/1900]
INFO:tensorflow:Evaluation [530/1900]
INFO:tensorflow:Evaluation [531/1900]
INFO:tensorf

INFO:tensorflow:Evaluation [722/1900]
INFO:tensorflow:Evaluation [723/1900]
INFO:tensorflow:Evaluation [724/1900]
INFO:tensorflow:Evaluation [725/1900]
INFO:tensorflow:Evaluation [726/1900]
INFO:tensorflow:Evaluation [727/1900]
INFO:tensorflow:Evaluation [728/1900]
INFO:tensorflow:Evaluation [729/1900]
INFO:tensorflow:Evaluation [730/1900]
INFO:tensorflow:Evaluation [731/1900]
INFO:tensorflow:Evaluation [732/1900]
INFO:tensorflow:Evaluation [733/1900]
INFO:tensorflow:Evaluation [734/1900]
INFO:tensorflow:Evaluation [735/1900]
INFO:tensorflow:Evaluation [736/1900]
INFO:tensorflow:Evaluation [737/1900]
INFO:tensorflow:Evaluation [738/1900]
INFO:tensorflow:Evaluation [739/1900]
INFO:tensorflow:Evaluation [740/1900]
INFO:tensorflow:Evaluation [741/1900]
INFO:tensorflow:Evaluation [742/1900]
INFO:tensorflow:Evaluation [743/1900]
INFO:tensorflow:Evaluation [744/1900]
INFO:tensorflow:Evaluation [745/1900]
INFO:tensorflow:Evaluation [746/1900]
INFO:tensorflow:Evaluation [747/1900]
INFO:tensorf

...

INFO:tensorflow:Evaluation [1150/1900]
INFO:tensorflow:Evaluation [1151/1900]
INFO:tensorflow:Evaluation [1152/1900]
INFO:tensorflow:Evaluation [1153/1900]
INFO:tensorflow:Evaluation [1154/1900]
INFO:tensorflow:Evaluation [1155/1900]
INFO:tensorflow:Evaluation [1156/1900]
INFO:tensorflow:Evaluation [1157/1900]
INFO:tensorflow:Evaluation [1158/1900]
INFO:tensorflow:Evaluation [1159/1900]
INFO:tensorflow:Evaluation [1160/1900]
INFO:tensorflow:Evaluation [1161/1900]
INFO:tensorflow:Evaluation [1162/1900]
INFO:tensorflow:Evaluation [1163/1900]
INFO:tensorflow:Evaluation [1164/1900]
INFO:tensorflow:Evaluation [1165/1900]
INFO:tensorflow:Evaluation [1166/1900]
INFO:tensorflow:Evaluation [1167/1900]
INFO:tensorflow:Evaluation [1168/1900]
INFO:tensorflow:Evaluation [1169/1900]
INFO:tensorflow:Evaluation [1170/1900]
INFO:tensorflow:Evaluation [1171/1900]
INFO:tensorflow:Evaluation [1172/1900]
INFO:tensorflow:Evaluation [1173/1900]
INFO:tensorflow:Evaluation [1174/1900]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1361/1900]
INFO:tensorflow:Evaluation [1362/1900]
INFO:tensorflow:Evaluation [1363/1900]
INFO:tensorflow:Evaluation [1364/1900]
INFO:tensorflow:Evaluation [1365/1900]
INFO:tensorflow:Evaluation [1366/1900]
INFO:tensorflow:Evaluation [1367/1900]
INFO:tensorflow:Evaluation [1368/1900]
INFO:tensorflow:Evaluation [1369/1900]
INFO:tensorflow:Evaluation [1370/1900]
INFO:tensorflow:Evaluation [1371/1900]
INFO:tensorflow:Evaluation [1372/1900]
INFO:tensorflow:Evaluation [1373/1900]
INFO:tensorflow:Evaluation [1374/1900]
INFO:tensorflow:Evaluation [1375/1900]
INFO:tensorflow:Evaluation [1376/1900]
INFO:tensorflow:Evaluation [1377/1900]
INFO:tensorflow:Evaluation [1378/1900]
INFO:tensorflow:Evaluation [1379/1900]
INFO:tensorflow:Evaluation [1380/1900]
INFO:tensorflow:Evaluation [1381/1900]
INFO:tensorflow:Evaluation [1382/1900]
INFO:tensorflow:Evaluation [1383/1900]
INFO:tensorflow:Evaluation [1384/1900]
INFO:tensorflow:Evaluation [1385/1900]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1572/1900]
INFO:tensorflow:Evaluation [1573/1900]
INFO:tensorflow:Evaluation [1574/1900]
INFO:tensorflow:Evaluation [1575/1900]
INFO:tensorflow:Evaluation [1576/1900]
INFO:tensorflow:Evaluation [1577/1900]
INFO:tensorflow:Evaluation [1578/1900]
INFO:tensorflow:Evaluation [1579/1900]
INFO:tensorflow:Evaluation [1580/1900]
INFO:tensorflow:Evaluation [1581/1900]
INFO:tensorflow:Evaluation [1582/1900]
INFO:tensorflow:Evaluation [1583/1900]
INFO:tensorflow:Evaluation [1584/1900]
INFO:tensorflow:Evaluation [1585/1900]
INFO:tensorflow:Evaluation [1586/1900]
INFO:tensorflow:Evaluation [1587/1900]
INFO:tensorflow:Evaluation [1588/1900]
INFO:tensorflow:Evaluation [1589/1900]
INFO:tensorflow:Evaluation [1590/1900]
INFO:tensorflow:Evaluation [1591/1900]
INFO:tensorflow:Evaluation [1592/1900]
INFO:tensorflow:Evaluation [1593/1900]
INFO:tensorflow:Evaluation [1594/1900]
INFO:tensorflow:Evaluation [1595/1900]
INFO:tensorflow:Evaluation [1596/1900]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1783/1900]
INFO:tensorflow:Evaluation [1784/1900]
INFO:tensorflow:Evaluation [1785/1900]
INFO:tensorflow:Evaluation [1786/1900]
INFO:tensorflow:Evaluation [1787/1900]
INFO:tensorflow:Evaluation [1788/1900]
INFO:tensorflow:Evaluation [1789/1900]
INFO:tensorflow:Evaluation [1790/1900]
INFO:tensorflow:Evaluation [1791/1900]
INFO:tensorflow:Evaluation [1792/1900]
INFO:tensorflow:Evaluation [1793/1900]
INFO:tensorflow:Evaluation [1794/1900]
INFO:tensorflow:Evaluation [1795/1900]
INFO:tensorflow:Evaluation [1796/1900]
INFO:tensorflow:Evaluation [1797/1900]
INFO:tensorflow:Evaluation [1798/1900]
INFO:tensorflow:Evaluation [1799/1900]
INFO:tensorflow:Evaluation [1800/1900]
INFO:tensorflow:Evaluation [1801/1900]
INFO:tensorflow:Evaluation [1802/1900]
INFO:tensorflow:Evaluation [1803/1900]
INFO:tensorflow:Evaluation [1804/1900]
INFO:tensorflow:Evaluation [1805/1900]
INFO:tensorflow:Evaluation [1806/1900]
INFO:tensorflow:Evaluation [1807/1900]
INFO:tensorflow:Evaluatio

Test Accuracy by Scikit-learn:  0.809473684211
CPU times: user 9min 3s, sys: 50 s, total: 9min 53s
Wall time: 1min 45s


###### CNN classifier

In [69]:
%%time

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

sess = tf.InteractiveSession()

COL_OUTCOME = 'class'
COL_FEATURE = [col for col in list(df['train'].columns) if col != COL_OUTCOME]

# cls2num = {cls:ind for (ind, cls) in enumerate(df['train']['class'].unique())}

count_feature = len(COL_FEATURE)
count_class = len(df['train']['class'].unique())

x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y_ = tf.placeholder(tf.float32, shape=[None, count_class], name='y_')

W = tf.Variable(tf.zeros([count_feature, count_class]))
b = tf.Variable(tf.zeros([count_class]))
y = tf.matmul(x, W) + b

# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_text = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_text, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32, name='keep_prob')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, count_class])
b_fc2 = bias_variable([count_class])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

print("CNN initialization finished")

CNN initialization finished
CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 36.7 ms


In [70]:
%%time

### Start to traini and evaluate the model

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

x_input = df_new['train']['x']
x_input = [np.array([
            np.float32(x_input.iloc[i].values)
        ])
    for i in range(x_input.shape[0])]
y_input = df_new['train']['y']
y_input = [np.array([
            np.float32(y_input.iloc[i].values)
        ])
    for i in range(y_input.shape[0])]
# y_input = [np.array([y_input.iloc[i].values]) for i in range(y_input.shape[0])]

# not use random input

for i in range(df['train'].shape[0] - 50):
    if 0 == i % 100:
        train_accuracy = []
        for j in range(50):
            train_accuracy.append(accuracy.eval(feed_dict={
                    keep_prob: 1,
                    x:  np.array([elem[0] for elem in x_input[i+j:i+j+50]]),#x_input.iloc[i+j].values, #
                    y_: np.array([elem[0] for elem in y_input[i+j:i+j+50]])#y_input.iloc[i+j].values #
                })
            )
        print("step {}, training accuracy {}".format(i, np.mean(train_accuracy)))
    train_step.run(feed_dict={
        keep_prob: 0.5,
        x:  np.array([elem[0] for elem in x_input[i:i+50]]),#x_input.iloc[i].values, #
        y_: np.array([elem[0] for elem in y_input[i:i+50]])#y_input.iloc[i].values#
    })

print("CNN training finished")

step 0, training accuracy 0.211600005627
step 100, training accuracy 0.321599990129
step 200, training accuracy 0.512800037861
step 300, training accuracy 0.61879992485
step 400, training accuracy 0.57279998064
step 500, training accuracy 0.709200084209
step 600, training accuracy 0.688800036907
step 700, training accuracy 0.630800008774
step 800, training accuracy 0.652400076389
step 900, training accuracy 0.734400093555
step 1000, training accuracy 0.644799947739
step 1100, training accuracy 0.745999932289
step 1200, training accuracy 0.767199933529
step 1300, training accuracy 0.762399971485
step 1400, training accuracy 0.737200081348
step 1500, training accuracy 0.761199951172
step 1600, training accuracy 0.827199995518
step 1700, training accuracy 0.82959997654
step 1800, training accuracy 0.682000100613
step 1900, training accuracy 0.841599881649
step 2000, training accuracy 0.853199899197
step 2100, training accuracy 0.77480006218
step 2200, training accuracy 0.798800051212
step

In [71]:
%%time

# Evaluate

x_input = df_new['test']['x']#df_new['test']['x']
x_input = [np.array([
            np.float32(x_input.iloc[i].values)
        ])
    for i in range(x_input.shape[0])]
y_input = df_new['test']['y']#df_new['test']['y']
y_input = [np.array([
            np.float32(y_input.iloc[i].values)
        ])
    for i in range(y_input.shape[0])]

for i in range(df['test'].shape[0] - 50):
    if 0 == i % 100:
        train_accuracy = []
        for j in range(50):
            train_accuracy.append(accuracy.eval(feed_dict={
                    keep_prob: 1,
                    x:  np.array([elem[0] for elem in x_input[i+j:i+j+50]]),#x_input.iloc[i+j].values, #
                    y_: np.array([elem[0] for elem in y_input[i+j:i+j+50]])#y_input.iloc[i+j].values #
                })
            )
        print("step {}, testing accuracy {}".format(i, np.mean(train_accuracy)))

        
print("CNN testing finished")

step 0, testing accuracy 0.712000131607
step 100, testing accuracy 0.715600073338
step 200, testing accuracy 0.726800024509
step 300, testing accuracy 0.857200026512
step 400, testing accuracy 0.729999899864
step 500, testing accuracy 0.82439994812
step 600, testing accuracy 0.726400017738
step 700, testing accuracy 0.704400002956
step 800, testing accuracy 0.795599997044
step 900, testing accuracy 0.645599961281
step 1000, testing accuracy 0.723200082779
step 1100, testing accuracy 0.737999975681
step 1200, testing accuracy 0.807200074196
step 1300, testing accuracy 0.759599983692
step 1400, testing accuracy 0.709999978542
step 1500, testing accuracy 0.768800020218
step 1600, testing accuracy 0.777999937534
step 1700, testing accuracy 0.814000070095
step 1800, testing accuracy 0.771599888802
CNN testing finished
CPU times: user 2min 40s, sys: 10.1 s, total: 2min 51s
Wall time: 26 s


##### 3.2.1.2 只用待学习语料建模

In [73]:
embedFrom = 'corpus'

In [85]:
%%time

# collect sentences from raw data
sentences = {}
pathtmp = {}
pathtmp['root'] = os.path.join(paths['dir.dataroot'], 'trialdata')
for tpart in ['train', 'test']:
    pathtmp[tpart] = os.path.join(pathtmp['root'], tpart)
    sentences[tpart] = []
    folderList = os.listdir(pathtmp[tpart])
    for folder in folderList:
        fileList = os.listdir(os.path.join(pathtmp[tpart], folder))
        for eachf in fileList:
            fpathtmp = os.path.join(pathtmp[tpart], folder, eachf)
            with open(fpathtmp, 'r') as f:
                sentences[tpart].append(f.read())
      #save sentences in file
        sentencePath = os.path.join(pathtmp['root'], 'sentences-{}'.format(tpart))
        with open(sentencePath, 'w') as f:
            for sentence in sentences[tpart]:
                f.write(sentence)
                f.write('\n')

pathtmp = os.path.join(pathtmp['root'], 'sentences-train')
sentences = []
with open(pathtmp, 'r') as f:
    buff = f.read()
    sentencesBuffer = buff.split('\n')
    sentences = [stcbuffer.split() for stcbuffer in sentencesBuffer]

print('get sentences from training corpus successfully')
print('example:')
print(sentences[random.randrange(len(sentences))])

get sentences from training corpus successfully
example:
['from', 'livesey', 'solntze', 'wpd', 'sgi', 'com', 'jon', 'livesey', 'subject', 're', 'slavery', 'was', 're', 'why', 'is', 'sex', 'only', 'allowed', 'in', 'marriage', 'organization', 'sgi', 'lines', 'three', 'seven', 'distribution', 'world', 'nntp', 'posting', 'host', 'solntze', 'wpd', 'sgi', 'com', 'in', 'article', 'mas', 'cadence', 'com', 'masud', 'khan', 'writes', 'leonard', 'i', 'll', 'give', 'you', 'an', 'example', 'of', 'this', 'my', 'father', 'recently', 'bought', 'a', 'business', 'the', 'business', 'price', 'was', 'one', 'five', 'zero', 'zero', 'zero', 'zero', 'pounds', 'and', 'my', 'father', 'approached', 'the', 'people', 'in', 'the', 'community', 'for', 'help', 'he', 'raised', 'six', 'zero', 'zero', 'zero', 'zero', 'pounds', 'in', 'interest', 'free', 'loans', 'from', 'friends', 'and', 'relatives', 'and', 'muslims', 'he', 'knew', 'five', 'zero', 'zero', 'zero', 'zero', 'had', 'cash', 'and', 'the', 'rest', 'he', 'got', '

In [87]:
%%time

vecsize = 784
model = gensim.models.word2vec.Word2Vec(iter=15, size=vecsize, sg=1, window=5, workers=12)
print("Model created.")

Model created.
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 238 µs


In [88]:
%%time
   
trimmer = lambda word, count, min_count: gensim.utils.RULE_DISCARD if word in stopwords else gensim.utils.RULE_KEEP

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


In [89]:
%%time

model.build_vocab(sentences=sentences, trim_rule=trimmer)

print("Vocabulary creation finished.")

Vocabulary creation finished.
CPU times: user 5.72 s, sys: 20 ms, total: 5.74 s
Wall time: 5.64 s


In [90]:
%%time

model.train(sentences=sentences)

print('Training finished.')

Training finished.
CPU times: user 3min 27s, sys: 56 ms, total: 3min 27s
Wall time: 26.8 s


In [91]:
%%time

# save model

calander = datetime.date.today().timetuple()

modelpath = os.path.join(
    paths['dir.{}.{}'.format(modelFrom, embedFrom)],
    '{}.{}.{}{}{}'.format(modelFrom, embedFrom, calander.tm_year, calander.tm_mon, calander.tm_mday)
)
model.save(modelpath)

print("save model finished")

save model finished
CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 12.4 ms


In [92]:
%%time

# Step 2: read data and save it in data['vec.train'] 和 data['vec.test']

data = {}
data['vec.train'] = {'w2v.mean':[], 'class':[]}
data['vec.test'] = {'w2v.mean':[], 'class':[]}

for tpart in ['train', 'test']:
    dirpath = paths['dir.{}'.format(tpart)]
    for (ind, cls) in enumerate(os.listdir(dirpath)):
        clspath = os.path.join(dirpath, cls)
        files = os.listdir(clspath)
        for f in files:
            fpath = os.path.join(clspath, f)
            with open(fpath, 'r') as readf:
                tokens = [token for token in readf.read().split()] # if token not in stopwords]#readf.read().split()#
                # Word2Vec representation
                # begin
                vec = np.array([0.0 for i in range(vecsize)])
                expectationVal = np.array([0.0 for i in range(vecsize)])
                countvec = 0
                for token in tokens:
                    try:
                        vec += model[token]
                        countvec += 1
                    except:
                        vec += expectationVal
                vec = vec / float(countvec)#float(len(tokens))
                 # end
            data['vec.{}'.format(tpart)]['w2v.mean'].append(vec)
            data['vec.{}'.format(tpart)]['class'].append(cls)

    tmp = data['vec.{}'.format(tpart)]
    ind = (random.sample(range(len(tmp['class'])), 1))[0]
    print("sample(transformed) from {}[{}]:\n[corpus]\n {}\n[class]\n{}".format(
            tpart, ind, tmp['w2v.mean'][ind], tmp['class'][ind]
        )
    )
    print()
    
print("Step 2 Succeed")

sample(transformed) from train[2137]:
[corpus]
 [ -6.92075080e-02  -1.10341109e-02  -1.57756309e-03  -4.38288346e-02
  -2.56870093e-03  -1.91868105e-02   2.59707384e-02  -1.38480009e-02
   4.66735698e-02   1.22320529e-02  -4.48232980e-03   3.01153942e-02
   4.60493026e-03   3.00879649e-02   4.19246667e-02   1.45628062e-02
  -5.77818627e-02   2.03765628e-02   2.11208692e-03   1.31738036e-02
   1.65932280e-03  -1.35084295e-02  -2.44918113e-02  -3.92084046e-02
  -5.48109850e-03  -4.01681491e-03   1.42943723e-02  -2.04842861e-03
   8.92381919e-03  -1.54173397e-02  -1.73201344e-04  -3.68162232e-02
   8.38746933e-03   1.47829359e-02   1.49441627e-03   1.45780508e-02
  -3.15133147e-02  -3.66009132e-02  -1.93746889e-02  -1.80645023e-02
  -6.49417490e-02  -2.29584450e-02  -2.50614714e-02  -1.47653579e-02
  -4.99981898e-04  -5.59790347e-02   2.02353659e-02   1.19156180e-03
  -6.45340259e-04  -6.39465189e-03  -4.33630103e-03   4.03514177e-03
   2.22452323e-02   7.41300533e-03  -7.16117787e-03  -5

sample(transformed) from test[1103]:
[corpus]
 [-0.11811182 -0.03760944  0.00637991 -0.10428453  0.02088733 -0.00743363
  0.00273809  0.00321351  0.07759132  0.02099297  0.06763557  0.07964662
  0.04184821  0.11546032  0.05451436 -0.00761499 -0.09659691  0.01623226
  0.00857963  0.02312686 -0.03373558  0.03801262 -0.01173902 -0.07968408
 -0.01005026 -0.01140222 -0.00076246  0.0052872   0.01264707 -0.04825866
  0.01448141 -0.0767749   0.07888501  0.05307805  0.00290516 -0.0237392
 -0.05275487 -0.0825643  -0.01165215 -0.06120615 -0.10834015 -0.03880486
 -0.02553512 -0.02379313 -0.00265643 -0.01575326  0.02981099 -0.05304489
 -0.03617571 -0.01693243 -0.01640014  0.00522742  0.01684704  0.09463365
  0.02964319  0.01782001  0.07333235  0.00321156  0.02612249 -0.03638247
  0.01153358  0.02508477  0.04155179  0.01252466  0.01814196  0.04597889
 -0.03922394  0.01359904 -0.00491344 -0.00076291  0.02965598 -0.04829625
 -0.03138166 -0.01471911 -0.01941514 -0.01868932  0.00892622 -0.0564291
  0.03

In [93]:
%%time

# Step 3: Save in Pandas.DataFrame
#
# 将 data['matrix.train'] 与 data['matrix.test'] 转换成 Pandas.DataFrame 格式，保存到 df['train'] 和 df['test'] 中（df 为字典格式：String -> DataFrame）

df = {}
csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')
for tpart in ['train', 'test']:
    datadict = {}
    datadict['class'] = data['vec.{}'.format(tpart)]['class']
    datavec = np.array(data['vec.{}'.format(tpart)]['w2v.mean'])
    for col in range(vecsize):
        datadict[col]= datavec[:, col]

    df[tpart] = pd.DataFrame(data=datadict)
    print("See df[{}]".format(tpart))
    display(df[tpart])
    print("\n\n\n")
    # write data in DataFrame into CSV
    csvpath = os.path.join(csvpath_root, '{}-w2v-{}-{}.csv'.format(tpart, embedFrom, modelFrom))
    df[tpart].to_csv(csvpath, columns=df[tpart].columns)
    
print("Step 3 Succeed.")

# 繁琐点：研究如何把 CSR 矩阵中的数据规整好放到 DataFrame 中，并与 Class 一一对应

See df[train]


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,-0.079730,-0.023649,-0.000637,-0.058202,0.000308,-0.016652,0.023493,-0.001410,0.078345,0.011816,...,-0.019484,-0.018181,0.052244,-0.027038,-0.025845,0.038562,0.051001,-0.001862,0.077121,soc.religion.christian
1,-0.056762,0.012930,0.021774,-0.066071,-0.015854,-0.005939,0.023844,0.020822,0.069722,-0.015533,...,-0.022090,-0.035425,0.057279,-0.005960,-0.040210,0.029291,0.056450,0.003017,0.062584,soc.religion.christian
2,-0.041504,-0.028601,-0.020247,-0.063503,-0.017398,-0.028696,0.034888,0.021110,0.054268,0.014402,...,-0.013897,-0.021263,0.051658,-0.011763,-0.060453,0.044683,0.044996,0.003645,0.053631,soc.religion.christian
3,-0.070549,-0.022650,0.004644,-0.057970,-0.013228,-0.018406,0.027923,0.011727,0.048545,-0.005384,...,-0.003719,-0.014226,0.046223,-0.005666,-0.023267,0.044336,0.045128,0.016655,0.063528,soc.religion.christian
4,-0.051495,-0.002146,-0.001128,-0.039290,0.003812,-0.021420,0.038133,-0.002986,0.042595,-0.002844,...,-0.026251,-0.023259,0.044939,-0.017648,-0.030844,0.038878,0.047336,-0.008602,0.066024,soc.religion.christian
5,-0.078105,0.016148,0.027750,-0.095598,-0.005802,-0.008280,0.010347,-0.029389,0.050281,-0.011014,...,-0.012819,-0.004744,0.052907,-0.029764,-0.015968,0.048749,0.035587,0.032801,0.059229,soc.religion.christian
6,-0.069843,-0.013937,-0.007243,-0.041216,-0.010258,-0.023068,0.035144,0.001942,0.050948,-0.000750,...,-0.007474,-0.024936,0.037026,-0.004318,-0.037518,0.038701,0.047312,-0.000626,0.049751,soc.religion.christian
7,-0.065727,0.025335,0.012322,-0.068324,0.001057,0.013478,0.009310,-0.007826,0.068203,-0.009847,...,-0.005894,-0.018491,0.063467,-0.034221,-0.025167,0.043036,0.057216,0.017436,0.056326,soc.religion.christian
8,-0.094956,0.002193,0.023108,-0.090306,0.003462,-0.021560,0.012234,-0.016177,0.068556,-0.004596,...,-0.008637,-0.022019,0.059442,-0.030542,-0.026341,0.048808,0.046653,0.014746,0.056790,soc.religion.christian
9,-0.062736,-0.001663,0.000600,-0.045071,0.000152,-0.020807,0.039427,0.012743,0.064969,-0.005784,...,-0.011838,-0.028456,0.051516,-0.013243,-0.024054,0.050275,0.044571,0.000135,0.050249,soc.religion.christian






See df[test]


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,-0.062496,-0.003992,0.009274,-0.059166,0.002295,-0.010985,0.022264,0.014221,0.069631,-0.004797,...,-0.015165,-0.017867,0.041458,-0.023496,-0.039388,0.032961,0.052439,-0.006439,0.065302,soc.religion.christian
1,-0.087414,-0.038249,0.007048,-0.076555,0.021722,-0.045024,0.028561,0.004363,0.065021,0.018761,...,-0.022608,-0.032574,0.034987,-0.024158,-0.033635,0.040352,0.059029,-0.004064,0.065818,soc.religion.christian
2,-0.061214,-0.011331,0.005822,-0.036661,0.003768,-0.026180,0.026175,-0.001027,0.057774,-0.004146,...,-0.035250,-0.032543,0.036931,-0.025581,-0.038027,0.036342,0.043178,-0.000201,0.075717,soc.religion.christian
3,-0.072991,0.003276,0.025641,-0.074216,0.004734,0.011358,0.000940,-0.004469,0.069140,-0.031780,...,-0.019148,-0.020766,0.044144,-0.021039,-0.032014,0.048216,0.056782,0.018681,0.063605,soc.religion.christian
4,-0.069047,-0.025650,-0.014127,-0.039385,-0.000390,-0.016138,0.033307,0.003785,0.049565,0.004687,...,-0.014008,-0.025831,0.031113,-0.008229,-0.027739,0.044987,0.045992,-0.009662,0.046029,soc.religion.christian
5,-0.053782,-0.001994,-0.000659,-0.049186,-0.004210,-0.010316,0.034825,0.019230,0.055765,-0.015361,...,-0.013282,-0.015699,0.045814,-0.005469,-0.026109,0.030754,0.040282,-0.009578,0.060240,soc.religion.christian
6,-0.078397,-0.027214,-0.019220,-0.039719,0.000998,-0.026669,0.037096,-0.003177,0.048374,-0.011524,...,-0.005910,-0.016200,0.049237,-0.010819,-0.024510,0.045075,0.039118,0.007250,0.061699,soc.religion.christian
7,-0.054396,-0.024790,-0.005471,-0.054194,-0.001723,-0.003584,0.037532,0.009406,0.062509,0.000281,...,-0.030041,-0.012802,0.036459,-0.016810,-0.029783,0.023699,0.036585,-0.013798,0.066923,soc.religion.christian
8,-0.046941,-0.021817,-0.002797,-0.035273,-0.014086,-0.034321,0.030448,-0.002469,0.047158,0.004280,...,-0.017431,-0.031450,0.057079,-0.013610,-0.037398,0.044692,0.050148,0.000716,0.060035,soc.religion.christian
9,-0.063387,-0.027813,-0.007046,-0.044512,-0.000325,-0.018768,0.028955,-0.001471,0.065737,-0.004054,...,-0.016120,-0.025815,0.050199,-0.014001,-0.032361,0.036760,0.040632,-0.007805,0.074645,soc.religion.christian






Step 3 Succeed.
CPU times: user 2.12 s, sys: 32 ms, total: 2.16 s
Wall time: 2.16 s


In [94]:
# if wanna read data from CSV file

df = {}

for tpart in ['train', 'test']:
    csvpath = os.path.join(
        csvpath_root, '{}-w2v-{}-{}.csv'.format(
            tpart, embedFrom, modelFrom
        )
    )
    if os.path.exists(csvpath):
        df[tpart] = pd.DataFrame.from_csv(csvpath)
        df[tpart] = df[tpart].sample(frac=1)
        df[tpart].reset_index(drop=True, inplace=True)
        print("read {} successfully".format(tpart))
        display(df[tpart])
        print('\n\n\n')

read train successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,-0.041930,-0.037320,-0.013703,-0.031316,0.000647,-0.041839,0.020111,-0.001131,0.030702,0.015909,...,-0.012208,-0.014173,0.034225,-0.030942,-0.060793,0.055482,0.032378,0.005133,0.042735,comp.graphics
1,-0.053875,-0.037723,-0.001289,-0.044911,-0.009387,-0.031085,0.022400,0.000807,0.055394,0.008216,...,-0.015606,-0.026844,0.035230,-0.013593,-0.040151,0.034606,0.047973,-0.007257,0.045528,alt.atheism
2,-0.082996,-0.006368,0.012487,-0.101907,0.026058,-0.006897,0.030774,0.009709,0.069928,-0.006884,...,-0.008285,-0.003806,0.053953,-0.033277,-0.027587,0.040798,0.049388,0.031073,0.056879,rec.motorcycles
3,-0.079253,-0.038878,-0.018226,-0.040129,0.012973,-0.007393,0.042378,0.015500,0.064230,-0.002062,...,-0.006541,-0.023788,0.039469,-0.031862,-0.025708,0.043924,0.047402,-0.004219,0.059046,alt.atheism
4,-0.098658,-0.062071,-0.013294,-0.096206,0.023424,-0.006853,0.051590,0.035089,0.062530,0.000769,...,-0.030714,-0.019996,0.063740,-0.029146,-0.017373,0.052743,0.052029,0.009889,0.070545,alt.atheism
5,-0.133948,-0.026668,0.026274,-0.126822,0.013906,-0.018019,0.007832,0.001418,0.102060,0.020701,...,-0.047264,-0.010577,0.065511,-0.047223,0.000912,0.063435,0.063396,0.009550,0.071706,rec.autos
6,-0.125721,-0.017746,0.000925,-0.110393,0.008402,0.008770,0.000420,-0.003976,0.060523,-0.011276,...,-0.017145,0.012790,0.061869,-0.043170,-0.009553,0.036598,0.040131,0.029338,0.024254,comp.graphics
7,-0.046582,-0.008309,-0.013047,-0.034686,-0.013737,-0.023121,0.038858,0.005264,0.042349,0.007917,...,-0.009330,-0.028854,0.057401,-0.013395,-0.021866,0.037975,0.050527,-0.002294,0.045047,soc.religion.christian
8,-0.104960,-0.037653,0.011340,-0.054826,0.034294,-0.034874,0.031261,0.015569,0.054510,0.006463,...,-0.022376,-0.023033,0.029968,-0.007767,-0.039972,0.035874,0.046882,0.012881,0.083525,rec.autos
9,-0.141489,-0.066759,0.001601,-0.105917,0.021009,0.006697,0.014165,0.018442,0.091156,0.000600,...,-0.025044,-0.002760,0.082174,-0.046818,-0.028289,0.065187,0.071128,0.026814,0.063396,rec.motorcycles






read test successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,-0.081731,-0.004764,-0.012101,-0.069220,-0.001379,-0.015316,0.022999,0.020267,0.069385,-0.006278,...,0.013480,-0.008676,0.056640,-0.040310,-0.022069,0.048490,0.034204,0.010040,0.065003,comp.graphics
1,-0.097792,-0.017958,0.001700,-0.055562,0.003669,-0.033278,0.033678,-0.004591,0.046991,-0.001492,...,-0.011707,-0.008645,0.051835,-0.019044,-0.026755,0.046990,0.042069,0.016343,0.057705,comp.graphics
2,-0.116530,0.008329,0.016162,-0.132806,0.017821,0.018096,-0.004903,-0.001694,0.083451,-0.009576,...,-0.014934,-0.007064,0.085981,-0.034492,-0.048087,0.089984,0.068729,0.002414,0.114820,rec.motorcycles
3,-0.106671,-0.017298,-0.018855,-0.059363,0.005475,-0.018509,0.002709,0.005687,0.056503,-0.024181,...,-0.007485,-0.000348,0.057278,-0.028655,-0.016583,0.034875,0.051682,0.024302,0.037784,comp.graphics
4,-0.065294,-0.013719,-0.000415,-0.044229,-0.001409,-0.023134,0.024657,0.002024,0.055189,-0.004980,...,-0.010764,-0.022938,0.039239,-0.010271,-0.025679,0.031539,0.047365,0.000367,0.048799,soc.religion.christian
5,-0.097884,-0.034024,0.035295,-0.085874,0.019200,-0.004469,0.028108,-0.003590,0.047012,-0.007293,...,-0.019252,-0.005879,0.061241,-0.027668,-0.014782,0.042143,0.051885,0.030211,0.033070,rec.motorcycles
6,-0.125321,-0.017656,0.024158,-0.124819,0.014406,0.013520,-0.003531,0.003272,0.110730,-0.018335,...,-0.027205,0.001510,0.084814,-0.056488,0.008497,0.043953,0.063471,0.022955,0.035609,alt.atheism
7,-0.087147,-0.012321,0.000941,-0.064898,0.009898,-0.007535,0.034204,0.007687,0.061507,-0.029589,...,-0.018586,0.004568,0.053618,-0.019018,-0.044329,0.046996,0.031444,0.032712,0.066138,comp.graphics
8,-0.088266,0.009463,0.006250,-0.092894,0.022079,0.011474,0.011496,0.009644,0.109954,-0.002389,...,-0.023825,-0.022800,0.073818,-0.041873,-0.034369,0.051517,0.071210,0.005400,0.059941,soc.religion.christian
9,-0.087879,-0.023612,0.004497,-0.080114,0.024681,-0.007263,0.026019,0.008766,0.046200,0.010603,...,-0.028790,-0.014976,0.051058,-0.036790,-0.010563,0.059333,0.055061,-0.001314,0.102799,rec.motorcycles


In [95]:
%%time

# if wanna read data from CSV file

df = {}

for tpart in ['train', 'test']:
    csvpath = os.path.join(
        csvpath_root, '{}-w2v-{}-{}.csv'.format(
            tpart, embedFrom, modelFrom
        )
    )
    if os.path.exists(csvpath):
        df[tpart] = pd.DataFrame.from_csv(csvpath)
        df[tpart] = df[tpart].sample(frac=1)
        df[tpart].reset_index(drop=True, inplace=True)
        print("read {} successfully".format(tpart))
        display(df[tpart])

read train successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,-0.094903,-0.033396,0.023281,-0.068885,0.012803,-0.020081,0.043820,-0.013916,0.045930,0.007536,...,0.015936,-0.009133,0.044235,-0.011449,-0.033533,0.049224,0.059578,-0.008464,0.073953,rec.autos
1,-0.098500,0.003244,0.054230,-0.109855,0.010125,0.003442,-0.017964,-0.000731,0.100552,-0.004235,...,-0.005796,-0.044728,0.058191,-0.029152,-0.057866,0.055716,0.082525,0.012761,0.089680,rec.autos
2,-0.139683,-0.053830,-0.005676,-0.095735,0.025591,-0.004752,0.049037,0.020243,0.075016,-0.003708,...,-0.013632,-0.001994,0.063991,-0.034154,-0.019847,0.055147,0.045145,0.033465,0.041801,rec.motorcycles
3,-0.112694,-0.026646,0.039184,-0.098880,0.024492,0.005924,0.010822,-0.010065,0.068318,0.020327,...,-0.034969,0.002065,0.055133,-0.046471,-0.006569,0.034517,0.063772,0.042543,0.047836,rec.motorcycles
4,-0.102097,0.016169,0.027686,-0.136797,0.000690,0.017346,-0.021475,-0.019313,0.093308,0.001078,...,-0.028761,0.011107,0.102111,-0.065938,-0.002415,0.051332,0.065822,0.018612,0.054210,comp.graphics
5,-0.070430,-0.021697,0.025177,-0.088297,0.017552,-0.004928,-0.013037,0.010590,0.092257,-0.027441,...,-0.015653,-0.017257,0.062221,-0.030948,-0.058316,0.065484,0.044497,-0.004387,0.106836,rec.autos
6,-0.179349,-0.019614,0.001812,-0.121889,0.023463,0.000710,0.033576,0.029739,0.109084,0.003450,...,-0.011901,-0.013323,0.070001,-0.018533,-0.030263,0.080563,0.057742,0.037061,0.049532,comp.graphics
7,-0.127285,-0.005624,0.015655,-0.119179,0.031591,0.046910,0.003071,-0.016354,0.086801,-0.019492,...,-0.052847,-0.000680,0.083374,-0.065415,-0.008515,0.047261,0.042404,0.035419,0.059149,rec.motorcycles
8,-0.089965,-0.017877,0.020218,-0.060343,0.010492,-0.026772,0.019693,-0.027253,0.058246,0.011499,...,-0.014743,-0.010517,0.052166,-0.021592,-0.039748,0.040507,0.046267,0.011517,0.087518,rec.motorcycles
9,-0.078160,-0.023529,0.002218,-0.040569,-0.012595,-0.004889,0.015876,-0.005516,0.055291,-0.011488,...,-0.000654,-0.016180,0.035070,-0.008485,-0.031401,0.056715,0.036504,-0.000791,0.055320,alt.atheism


read test successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,-0.117148,-0.021425,0.000099,-0.113312,0.024052,0.004719,0.000188,0.012720,0.083197,-0.005689,...,-0.033537,-0.013109,0.059935,-0.033495,-0.015420,0.052427,0.064892,0.036332,0.064498,rec.autos
1,-0.102021,-0.037374,0.000158,-0.067116,0.005629,-0.028177,0.034275,-0.017012,0.071624,-0.003065,...,-0.007797,-0.018987,0.035484,-0.020930,-0.038710,0.043458,0.037201,0.010973,0.079081,rec.autos
2,-0.089513,-0.022315,-0.005322,-0.054825,-0.000899,-0.023198,0.032110,-0.005001,0.061583,0.002839,...,-0.011607,-0.011517,0.045687,-0.018633,-0.039093,0.030294,0.044496,0.008770,0.062570,rec.autos
3,-0.102370,-0.018573,0.005848,-0.061515,0.010681,-0.016677,0.038279,-0.001188,0.054219,-0.010859,...,-0.016656,0.000728,0.068354,-0.041630,-0.023546,0.038867,0.051942,0.020903,0.065162,comp.graphics
4,-0.061355,0.017289,0.028212,-0.070635,-0.003494,0.007862,0.002878,-0.011259,0.062566,-0.024810,...,-0.012964,-0.020334,0.039886,-0.028835,-0.019112,0.025722,0.047179,0.003098,0.071323,soc.religion.christian
5,-0.069710,-0.007212,0.012999,-0.063925,-0.010738,-0.016342,-0.000341,0.002082,0.081737,-0.000286,...,-0.019473,-0.020042,0.057389,-0.014258,-0.044634,0.038760,0.048375,-0.002766,0.075653,alt.atheism
6,-0.105284,-0.018029,0.001020,-0.062485,-0.005475,-0.023145,0.021618,-0.002325,0.051883,-0.012287,...,0.005362,-0.009142,0.051488,-0.016278,-0.026575,0.043514,0.038242,0.020516,0.059948,comp.graphics
7,-0.090577,-0.045687,-0.028171,-0.042403,0.011147,-0.036393,0.021754,0.017434,0.044115,-0.000617,...,-0.003240,-0.017318,0.043766,-0.007699,-0.029511,0.048096,0.035694,0.024916,0.060409,comp.graphics
8,-0.067873,-0.023985,-0.012259,-0.046345,0.004283,-0.012247,0.035525,0.007658,0.066258,-0.006282,...,-0.014699,-0.018004,0.040865,-0.012956,-0.030815,0.038679,0.042452,-0.012858,0.055945,soc.religion.christian
9,-0.113428,-0.046355,-0.011424,-0.061250,0.017173,-0.013115,0.021826,0.002743,0.065482,0.001897,...,-0.003136,-0.017735,0.047614,-0.021488,-0.042859,0.052737,0.054281,0.010689,0.067286,comp.graphics


CPU times: user 864 ms, sys: 16 ms, total: 880 ms
Wall time: 877 ms


###### SVM classifier

In [96]:
%%time

# Step 5.1.1: SVM

# if 'TFIDF' == modelChoice:

#train
X_train = df['train'].drop('class', axis=1)
y_train = df['train']['class']
#test
X_test = df['test'].drop('class', axis=1)
y_test_true = df['test']['class']

# else:
#     #train
#     X_train = df_new['train']['x']
#     y_train = df_new['train']['y']
#     #test
#     X_test = df_new['test']['x']
#     y_test_true = df_new['test']['y']

clf = LinearSVC()
clf.fit(X_train, y_train)

print("Step 4 finished")

Step 4 finished
CPU times: user 1.24 s, sys: 4 ms, total: 1.25 s
Wall time: 1.25 s


In [97]:
%%time
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Step 5.1.2: Test
y_test_pred = clf.predict(X_test)
print(accuracy_score(y_test_true, y_test_pred))
print(f1_score(y_test_true, y_test_pred, average='macro'))
print(f1_score(y_test_true, y_test_pred, average='micro'))

0.827368421053
0.823115546239
0.827368421053
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 19.7 ms


###### DNN classifier

In [98]:
%%time

# Step 4: One-hot representation for labels

csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')

lb = LabelBinarizer()
lb.fit(df['train']['class'])

df_new = {}
for tpart in ['train', 'test']:
    labels = lb.transform(df[tpart]['class'])
    labelsDf = pd.DataFrame(labels, columns=["class-{}".format(i) for i in range(len(lb.classes_))])
    df_new[tpart] = {}
    df_new[tpart]['y'] = labelsDf
    df_new[tpart]['x'] = df[tpart].drop('class', axis=1)
    df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
    #save in CSV
    for subpart in ['x', 'y', 'all']:
        csvpath = os.path.join(csvpath_root, "{}-cleanLabels-{}-{}.csv".format(tpart, subpart, modelFrom))
        df_new[tpart][subpart].to_csv(csvpath)
    
print("label cleaning succussfully")

label cleaning succussfully
CPU times: user 3.7 s, sys: 52 ms, total: 3.75 s
Wall time: 3.75 s


In [99]:
%%time

## Step 5 : Train the classifier

COL_OUTCOME = 'class'
COL_FEATURE = [str(col) for col in list(df['train'].columns) if col != COL_OUTCOME]

cls2num = {cls:ind for (ind, cls) in enumerate(df['train']['class'].unique())}

def my_input_fn(dataset):
    # Save dataset in tf format
    feature_cols = {
        str(col): tf.constant(
            df[dataset][str(col)].values
        )
        for col in COL_FEATURE
    }
    labels = tf.constant([cls2num[labelname] for labelname in df[dataset][COL_OUTCOME].values])
    # Returns the feature columns and labels in tf format
    return feature_cols, labels

feature_columns = [tf.contrib.layers.real_valued_column(column_name=str(col)) for col in COL_FEATURE]
clf = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns, 
    hidden_units=[512], 
    n_classes=len(df['train']['class'].unique())
)

clf.fit(input_fn=lambda: my_input_fn('train'), steps=2000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1464dfc2d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpjRXwq8/model.ckpt.
INFO:tensorflow:loss = 1.60779, step = 1
INFO:tensorflow:global_step/sec: 11.3903
INFO:tensorflow:loss = 1.3959, step = 101
INFO:tensorflow:global_step/sec: 11.525
INFO:tensorflow:loss = 1.21157, step = 201
INFO:tensorflow:global_step/sec: 11.6707
INFO:tensorflow:loss = 1.07089, step = 301
INFO:tensorflow:global_step/sec: 11.8325
INFO:tensorflow:loss = 0.954396, step = 401
INFO:tensorflow:global_step/sec: 11.7476
INFO:tensorflow:loss = 0.863967, step = 501
INFO:tensorflow:global_step/sec: 11.6798
INFO:tensorflow:loss = 0.794398, step = 601
INFO:ten

In [100]:
%%time

## Step 6: Evaluate

accuracy_score = clf.evaluate(input_fn=lambda: my_input_fn('test'), steps=df['test'].shape[0])['accuracy']
print("Test Accuracy by TensorFlow: {}".format(accuracy_score))

X_tensor_test, yt = my_input_fn('test')
tensorPredCls = list(clf.predict(input_fn=lambda: my_input_fn('test')))
num2cls = {v:k for (k, v) in cls2num.items()}
tensorPredClsStr = [num2cls[i] for i in tensorPredCls]
y_test_true = df['test']['class']
print('Test Accuracy by Scikit-learn: ', f1_score(y_test_true, tensorPredClsStr, average='micro'))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-13-09:41:54
INFO:tensorflow:Evaluation [1/1900]
INFO:tensorflow:Evaluation [2/1900]
INFO:tensorflow:Evaluation [3/1900]
INFO:tensorflow:Evaluation [4/1900]
INFO:tensorflow:Evaluation [5/1900]
INFO:tensorflow:Evaluation [6/1900]
INFO:tensorflow:Evaluation [7/1900]
INFO:tensorflow:Evaluation [8/1900]
INFO:tensorflow:Evaluation [9/1900]
INFO:tensorflow:Evaluation [10/1900]
INFO:tensorflow:Evaluation [11/1900]
INFO:tensorflow:Evaluation [12/1900]
INFO:tensorflow:Evaluation [13/1900]
INFO:tensorflow:Evaluation [14/1900]
INFO:tensorflow:Evaluation [15/1900]
INFO:tensorflow:Evaluation [16/1900]
INFO:tensorflow:Evaluation [1

INFO:tensorflow:Evaluation [73/1900]
INFO:tensorflow:Evaluation [74/1900]
INFO:tensorflow:Evaluation [75/1900]
INFO:tensorflow:Evaluation [76/1900]
INFO:tensorflow:Evaluation [77/1900]
INFO:tensorflow:Evaluation [78/1900]
INFO:tensorflow:Evaluation [79/1900]
INFO:tensorflow:Evaluation [80/1900]
INFO:tensorflow:Evaluation [81/1900]
INFO:tensorflow:Evaluation [82/1900]
INFO:tensorflow:Evaluation [83/1900]
INFO:tensorflow:Evaluation [84/1900]
INFO:tensorflow:Evaluation [85/1900]
INFO:tensorflow:Evaluation [86/1900]
INFO:tensorflow:Evaluation [87/1900]
INFO:tensorflow:Evaluation [88/1900]
INFO:tensorflow:Evaluation [89/1900]
INFO:tensorflow:Evaluation [90/1900]
INFO:tensorflow:Evaluation [91/1900]
INFO:tensorflow:Evaluation [92/1900]
INFO:tensorflow:Evaluation [93/1900]
INFO:tensorflow:Evaluation [94/1900]
INFO:tensorflow:Evaluation [95/1900]
INFO:tensorflow:Evaluation [96/1900]
INFO:tensorflow:Evaluation [97/1900]
INFO:tensorflow:Evaluation [98/1900]
INFO:tensorflow:Evaluation [99/1900]
I

INFO:tensorflow:Evaluation [290/1900]
INFO:tensorflow:Evaluation [291/1900]
INFO:tensorflow:Evaluation [292/1900]
INFO:tensorflow:Evaluation [293/1900]
INFO:tensorflow:Evaluation [294/1900]
INFO:tensorflow:Evaluation [295/1900]
INFO:tensorflow:Evaluation [296/1900]
INFO:tensorflow:Evaluation [297/1900]
INFO:tensorflow:Evaluation [298/1900]
INFO:tensorflow:Evaluation [299/1900]
INFO:tensorflow:Evaluation [300/1900]
INFO:tensorflow:Evaluation [301/1900]
INFO:tensorflow:Evaluation [302/1900]
INFO:tensorflow:Evaluation [303/1900]
INFO:tensorflow:Evaluation [304/1900]
INFO:tensorflow:Evaluation [305/1900]
INFO:tensorflow:Evaluation [306/1900]
INFO:tensorflow:Evaluation [307/1900]
INFO:tensorflow:Evaluation [308/1900]
INFO:tensorflow:Evaluation [309/1900]
INFO:tensorflow:Evaluation [310/1900]
INFO:tensorflow:Evaluation [311/1900]
INFO:tensorflow:Evaluation [312/1900]
INFO:tensorflow:Evaluation [313/1900]
INFO:tensorflow:Evaluation [314/1900]
INFO:tensorflow:Evaluation [315/1900]
INFO:tensorf

INFO:tensorflow:Evaluation [506/1900]
INFO:tensorflow:Evaluation [507/1900]
INFO:tensorflow:Evaluation [508/1900]
INFO:tensorflow:Evaluation [509/1900]
INFO:tensorflow:Evaluation [510/1900]
INFO:tensorflow:Evaluation [511/1900]
INFO:tensorflow:Evaluation [512/1900]
INFO:tensorflow:Evaluation [513/1900]
INFO:tensorflow:Evaluation [514/1900]
INFO:tensorflow:Evaluation [515/1900]
INFO:tensorflow:Evaluation [516/1900]
INFO:tensorflow:Evaluation [517/1900]
INFO:tensorflow:Evaluation [518/1900]
INFO:tensorflow:Evaluation [519/1900]
INFO:tensorflow:Evaluation [520/1900]
INFO:tensorflow:Evaluation [521/1900]
INFO:tensorflow:Evaluation [522/1900]
INFO:tensorflow:Evaluation [523/1900]
INFO:tensorflow:Evaluation [524/1900]
INFO:tensorflow:Evaluation [525/1900]
INFO:tensorflow:Evaluation [526/1900]
INFO:tensorflow:Evaluation [527/1900]
INFO:tensorflow:Evaluation [528/1900]
INFO:tensorflow:Evaluation [529/1900]
INFO:tensorflow:Evaluation [530/1900]
INFO:tensorflow:Evaluation [531/1900]
INFO:tensorf

INFO:tensorflow:Evaluation [722/1900]
INFO:tensorflow:Evaluation [723/1900]
INFO:tensorflow:Evaluation [724/1900]
INFO:tensorflow:Evaluation [725/1900]
INFO:tensorflow:Evaluation [726/1900]
INFO:tensorflow:Evaluation [727/1900]
INFO:tensorflow:Evaluation [728/1900]
INFO:tensorflow:Evaluation [729/1900]
INFO:tensorflow:Evaluation [730/1900]
INFO:tensorflow:Evaluation [731/1900]
INFO:tensorflow:Evaluation [732/1900]
INFO:tensorflow:Evaluation [733/1900]
INFO:tensorflow:Evaluation [734/1900]
INFO:tensorflow:Evaluation [735/1900]
INFO:tensorflow:Evaluation [736/1900]
INFO:tensorflow:Evaluation [737/1900]
INFO:tensorflow:Evaluation [738/1900]
INFO:tensorflow:Evaluation [739/1900]
INFO:tensorflow:Evaluation [740/1900]
INFO:tensorflow:Evaluation [741/1900]
INFO:tensorflow:Evaluation [742/1900]
INFO:tensorflow:Evaluation [743/1900]
INFO:tensorflow:Evaluation [744/1900]
INFO:tensorflow:Evaluation [745/1900]
INFO:tensorflow:Evaluation [746/1900]
INFO:tensorflow:Evaluation [747/1900]
INFO:tensorf

...

INFO:tensorflow:Evaluation [1150/1900]
INFO:tensorflow:Evaluation [1151/1900]
INFO:tensorflow:Evaluation [1152/1900]
INFO:tensorflow:Evaluation [1153/1900]
INFO:tensorflow:Evaluation [1154/1900]
INFO:tensorflow:Evaluation [1155/1900]
INFO:tensorflow:Evaluation [1156/1900]
INFO:tensorflow:Evaluation [1157/1900]
INFO:tensorflow:Evaluation [1158/1900]
INFO:tensorflow:Evaluation [1159/1900]
INFO:tensorflow:Evaluation [1160/1900]
INFO:tensorflow:Evaluation [1161/1900]
INFO:tensorflow:Evaluation [1162/1900]
INFO:tensorflow:Evaluation [1163/1900]
INFO:tensorflow:Evaluation [1164/1900]
INFO:tensorflow:Evaluation [1165/1900]
INFO:tensorflow:Evaluation [1166/1900]
INFO:tensorflow:Evaluation [1167/1900]
INFO:tensorflow:Evaluation [1168/1900]
INFO:tensorflow:Evaluation [1169/1900]
INFO:tensorflow:Evaluation [1170/1900]
INFO:tensorflow:Evaluation [1171/1900]
INFO:tensorflow:Evaluation [1172/1900]
INFO:tensorflow:Evaluation [1173/1900]
INFO:tensorflow:Evaluation [1174/1900]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1361/1900]
INFO:tensorflow:Evaluation [1362/1900]
INFO:tensorflow:Evaluation [1363/1900]
INFO:tensorflow:Evaluation [1364/1900]
INFO:tensorflow:Evaluation [1365/1900]
INFO:tensorflow:Evaluation [1366/1900]
INFO:tensorflow:Evaluation [1367/1900]
INFO:tensorflow:Evaluation [1368/1900]
INFO:tensorflow:Evaluation [1369/1900]
INFO:tensorflow:Evaluation [1370/1900]
INFO:tensorflow:Evaluation [1371/1900]
INFO:tensorflow:Evaluation [1372/1900]
INFO:tensorflow:Evaluation [1373/1900]
INFO:tensorflow:Evaluation [1374/1900]
INFO:tensorflow:Evaluation [1375/1900]
INFO:tensorflow:Evaluation [1376/1900]
INFO:tensorflow:Evaluation [1377/1900]
INFO:tensorflow:Evaluation [1378/1900]
INFO:tensorflow:Evaluation [1379/1900]
INFO:tensorflow:Evaluation [1380/1900]
INFO:tensorflow:Evaluation [1381/1900]
INFO:tensorflow:Evaluation [1382/1900]
INFO:tensorflow:Evaluation [1383/1900]
INFO:tensorflow:Evaluation [1384/1900]
INFO:tensorflow:Evaluation [1385/1900]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1572/1900]
INFO:tensorflow:Evaluation [1573/1900]
INFO:tensorflow:Evaluation [1574/1900]
INFO:tensorflow:Evaluation [1575/1900]
INFO:tensorflow:Evaluation [1576/1900]
INFO:tensorflow:Evaluation [1577/1900]
INFO:tensorflow:Evaluation [1578/1900]
INFO:tensorflow:Evaluation [1579/1900]
INFO:tensorflow:Evaluation [1580/1900]
INFO:tensorflow:Evaluation [1581/1900]
INFO:tensorflow:Evaluation [1582/1900]
INFO:tensorflow:Evaluation [1583/1900]
INFO:tensorflow:Evaluation [1584/1900]
INFO:tensorflow:Evaluation [1585/1900]
INFO:tensorflow:Evaluation [1586/1900]
INFO:tensorflow:Evaluation [1587/1900]
INFO:tensorflow:Evaluation [1588/1900]
INFO:tensorflow:Evaluation [1589/1900]
INFO:tensorflow:Evaluation [1590/1900]
INFO:tensorflow:Evaluation [1591/1900]
INFO:tensorflow:Evaluation [1592/1900]
INFO:tensorflow:Evaluation [1593/1900]
INFO:tensorflow:Evaluation [1594/1900]
INFO:tensorflow:Evaluation [1595/1900]
INFO:tensorflow:Evaluation [1596/1900]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1783/1900]
INFO:tensorflow:Evaluation [1784/1900]
INFO:tensorflow:Evaluation [1785/1900]
INFO:tensorflow:Evaluation [1786/1900]
INFO:tensorflow:Evaluation [1787/1900]
INFO:tensorflow:Evaluation [1788/1900]
INFO:tensorflow:Evaluation [1789/1900]
INFO:tensorflow:Evaluation [1790/1900]
INFO:tensorflow:Evaluation [1791/1900]
INFO:tensorflow:Evaluation [1792/1900]
INFO:tensorflow:Evaluation [1793/1900]
INFO:tensorflow:Evaluation [1794/1900]
INFO:tensorflow:Evaluation [1795/1900]
INFO:tensorflow:Evaluation [1796/1900]
INFO:tensorflow:Evaluation [1797/1900]
INFO:tensorflow:Evaluation [1798/1900]
INFO:tensorflow:Evaluation [1799/1900]
INFO:tensorflow:Evaluation [1800/1900]
INFO:tensorflow:Evaluation [1801/1900]
INFO:tensorflow:Evaluation [1802/1900]
INFO:tensorflow:Evaluation [1803/1900]
INFO:tensorflow:Evaluation [1804/1900]
INFO:tensorflow:Evaluation [1805/1900]
INFO:tensorflow:Evaluation [1806/1900]
INFO:tensorflow:Evaluation [1807/1900]
INFO:tensorflow:Evaluatio

Test Accuracy by Scikit-learn:  0.818421052632
CPU times: user 9min 1s, sys: 50.7 s, total: 9min 51s
Wall time: 1min 45s


###### CNN classifier

In [101]:
%%time

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

sess = tf.InteractiveSession()

COL_OUTCOME = 'class'
COL_FEATURE = [col for col in list(df['train'].columns) if col != COL_OUTCOME]

# cls2num = {cls:ind for (ind, cls) in enumerate(df['train']['class'].unique())}

count_feature = len(COL_FEATURE)
count_class = len(df['train']['class'].unique())

x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y_ = tf.placeholder(tf.float32, shape=[None, count_class], name='y_')

W = tf.Variable(tf.zeros([count_feature, count_class]))
b = tf.Variable(tf.zeros([count_class]))
y = tf.matmul(x, W) + b

# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_text = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_text, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32, name='keep_prob')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, count_class])
b_fc2 = bias_variable([count_class])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

print("CNN initialization finished")

CNN initialization finished
CPU times: user 36 ms, sys: 4 ms, total: 40 ms
Wall time: 39 ms


In [102]:
%%time

### Start to traini and evaluate the model

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

x_input = df_new['train']['x']
x_input = [np.array([
            np.float32(x_input.iloc[i].values)
        ])
    for i in range(x_input.shape[0])]
y_input = df_new['train']['y']
y_input = [np.array([
            np.float32(y_input.iloc[i].values)
        ])
    for i in range(y_input.shape[0])]
# y_input = [np.array([y_input.iloc[i].values]) for i in range(y_input.shape[0])]

# not use random input

for i in range(df['train'].shape[0] - 50):
    if 0 == i % 100:
        train_accuracy = []
        for j in range(50):
            train_accuracy.append(accuracy.eval(feed_dict={
                    keep_prob: 1,
                    x:  np.array([elem[0] for elem in x_input[i+j:i+j+50]]),#x_input.iloc[i+j].values, #
                    y_: np.array([elem[0] for elem in y_input[i+j:i+j+50]])#y_input.iloc[i+j].values #
                })
            )
        print("step {}, training accuracy {}".format(i, np.mean(train_accuracy)))
    train_step.run(feed_dict={
        keep_prob: 0.5,
        x:  np.array([elem[0] for elem in x_input[i:i+50]]),#x_input.iloc[i].values, #
        y_: np.array([elem[0] for elem in y_input[i:i+50]])#y_input.iloc[i].values#
    })

print("CNN training finished")

step 0, training accuracy 0.175200000405
step 100, training accuracy 0.474399983883
step 200, training accuracy 0.585599958897
step 300, training accuracy 0.564399957657
step 400, training accuracy 0.557599961758
step 500, training accuracy 0.614799976349
step 600, training accuracy 0.706399977207
step 700, training accuracy 0.64359998703
step 800, training accuracy 0.70759999752
step 900, training accuracy 0.7675999403
step 1000, training accuracy 0.782399892807
step 1100, training accuracy 0.740800023079
step 1200, training accuracy 0.716000080109
step 1300, training accuracy 0.711199998856
step 1400, training accuracy 0.87640017271
step 1500, training accuracy 0.841600060463
step 1600, training accuracy 0.770799994469
step 1700, training accuracy 0.743200004101
step 1800, training accuracy 0.870800018311
step 1900, training accuracy 0.76239991188
step 2000, training accuracy 0.836400151253
step 2100, training accuracy 0.773999929428
step 2200, training accuracy 0.790799915791
step 2

In [103]:
%%time

# Evaluate

x_input = df_new['test']['x']#df_new['test']['x']
x_input = [np.array([
            np.float32(x_input.iloc[i].values)
        ])
    for i in range(x_input.shape[0])]
y_input = df_new['test']['y']#df_new['test']['y']
y_input = [np.array([
            np.float32(y_input.iloc[i].values)
        ])
    for i in range(y_input.shape[0])]

for i in range(df['test'].shape[0] - 50):
    if 0 == i % 100:
        train_accuracy = []
        for j in range(50):
            train_accuracy.append(accuracy.eval(feed_dict={
                    keep_prob: 1,
                    x:  np.array([elem[0] for elem in x_input[i+j:i+j+50]]),#x_input.iloc[i+j].values, #
                    y_: np.array([elem[0] for elem in y_input[i+j:i+j+50]])#y_input.iloc[i+j].values #
                })
            )
        print("step {}, testing accuracy {}".format(i, np.mean(train_accuracy)))

        
print("CNN testing finished")

step 0, testing accuracy 0.698399960995
step 100, testing accuracy 0.814400017262
step 200, testing accuracy 0.720800101757
step 300, testing accuracy 0.780399918556
step 400, testing accuracy 0.846400022507
step 500, testing accuracy 0.793200075626
step 600, testing accuracy 0.743200063705
step 700, testing accuracy 0.833999931812
step 800, testing accuracy 0.732000052929
step 900, testing accuracy 0.76919990778
step 1000, testing accuracy 0.717200040817
step 1100, testing accuracy 0.86000007391
step 1200, testing accuracy 0.784800112247
step 1300, testing accuracy 0.80119997263
step 1400, testing accuracy 0.761600017548
step 1500, testing accuracy 0.79239988327
step 1600, testing accuracy 0.782799899578
step 1700, testing accuracy 0.815999925137
step 1800, testing accuracy 0.792400062084
CNN testing finished
CPU times: user 2min 43s, sys: 10 s, total: 2min 53s
Wall time: 27.1 s


#### 3.2.2 TensorFlow 训练

In [ ]:
modelFrom = 'TensorFlow'

##### 3.2.1.1 基于 text8 建模

##### 3.2.1.2 只用待学习语料建模

## 总结

### 1. 对于同一种分类器训练法，不同表示模型对结果的影响

### 2. 对于同一种表示模型，不同训练模型对结果的影响

### 3. 综合来看，「表示模型 + 分类器」组合的效果评估

### 4. 展望